## Note: Please ensure you run this workbook from the same folder as where the supplied data files are stored in your computer.

# Preparing the Environment for Exploratory Data Analysis

In order for the work to be reproduceable, before running this notebook, please ensure that the requisite libraries/modules are installed on the local machine. In particular, amongst some of the libraries that are being used here which were not covered in the course can be installed by running the pip install cell below.

There may be other libraries or modules are being used here which will need to be installed locally. Please go through full list of models and libraries to ensure that the most recent versions of all of them are installed.

It is possible to **skip this section in part or whole if some or all the libraries listed below in the pip install command cell are already installed** on the local machine. 

Additionally, please ensure that all the datafiles supplied with this notebook and the underlying raw data from TfL supplied by LSE/TW are located in the same directory as where this notebook is saved on.

### Coding Methodology

Variable (Dataframe) names, such as df, df1..., have been reused in most of the cases to keep memory and resource allocation in this jupyter notebook to a minimum.

In [ ]:
# Installing additional libraries.
# It is possible that this take some time depending on your local machine and internet connection.
!pip install calmap
!pip install gapminder
!pip install plotly_calplot
!pip install ipywidgets
!pip install regex
!pip install pywaffle
!pip install calplot
!pip install plotly
!pip install --upgrade autopep8
!pip install pycodestyle
!pip install --upgrade pycodestyle
!pip uninstall pycodestyle

### Datafiles supplied with this Notebook

**Twitter scrapes:**

- twitter_scrape_071022en.csv
- twitter_scrape_081022en.csv
- twitter_scrape_091022en.csv
- twitter_scrape_101022en.csv
- twitter_scrape_111022en.csv
- twitter_scrape_111022ena.csv
- twitter_scrape_121022en.csv
- twitter_scrape_131022en.csv
- twitter_scrape_131022ena.csv
- twitter_scrape_131022enb.csv
- twitter_scrape_131022enc.csv
- twitter_scrape_141022en.csv
- twitter_scrape_151022en.csv
- twitter_scrape_151022enb.csv
- twitter_scrape_161022en.csv
- twitter_scrape_161022ena.csv
- twitter_scrape_171022en.csv
- twitter_scrape_191022en.csv
- user.csv

**Other External Data:**
- London_population.csv
- New_York_population.csv
- Sydney_population.csv
- private_cars_london.csv
- traffic_flow_borough.csv
- London_Biking_sites_reconv.xlsx - This file contains the same information as the supplied data on London's cycle parking sites except that the "Eastings" & "Northings" data has been converted into the more conventional "Longitude" & "Latitude".
- cycle_parking.csv
- restricted_point.csv
- signage.csv
- signal.csv
- traffic_calming.csv
- advanced_stop_line.csv
- restricted_route.csv
- crossing.csv
- cycle_lane_track.csv
- TFL Cycle Hire 2017.csv

**Underlying CSV Datafiles provided by LSE/ThoughtWorks are required but not supplied with this Notebook.**

## Importing required packages

In [ ]:
# Import libraries & set up date parsing functionality.
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm 
import nltk
import os
import string   
import re
import math
import squarify
import plotly.express as px
import requests
import functools as ft
import calmap
import calplot
import locale

# Copy your user specific YAML file and Twitter keys over to the same folder,
# as the location of this Jupyter Notebook before you start to work.
import yaml
from yaml.loader import SafeLoader
from twitter import *

# Import modules.
from gapminder import gapminder
from pywaffle import Waffle
from datetime import datetime
from statsmodels.formula.api import ols
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
from collections import Counter
from plotly_calplot import calplot
from scipy.ndimage import gaussian_gradient_magnitude
from wordcloud import WordCloud, ImageColorGenerator
from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import warnings
warnings.filterwarnings('ignore')

# Setting up a date parser using a private funciton, lambda.
# This will convert dates into a format required for aggregation & indexation.
d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p')

# Default settings for matplotlibs to be used throughout
plt.rcParams["axes.facecolor"] = "w"

# Setting up locale for Mac Users

Run this code only if the local machine is a Mac. Skip this if the machine is Windows.

In [ ]:
# Create local setting for datetime parser to GB.
locale.setlocale(locale.LC_ALL, 'en_GB')

# Exploring Twitter Data about Cycling

Is it at all worthwhile to look at cycling in London? Asides the traditional data that has been supplied, the task here is to extract the most recent posts on Twitter about Cycling. From these, the objectives are:
- Discover locational information around where the topic generates the most social media content from 
- To see if any of the three cities being analysed are amongst these locations
- Get an overview of the sentiment expressed in such content on the topic
- Get an overview of the most common words used in such content to discover any underlying themes that can be explored further.

The output of this analysis using social data can be used as supplementary evidence to partially inform the hypothesis that Londoner's can be influenced to increase the uptake of cycling amongst them. 

**Note** some of the outputs in this section will change very slightly versus the notes as the underlying data will be marginally refreshed each time the work book is run. This cannot be accounted for in the analysis and accompanying insights. However, it is not expected that either the analysis or insights will change significantly on the basis of this marginal refresh of the underlying data.

In [ ]:
# Copy the YAML file and Twitter keys over to this Jupyter Notebook before you start to work.
# Import the yaml file - remember to specify the whole path and use / between directories.
# The name of your yaml file will need to be inserted.
# Here the yaml file on the local machine is called twitter.yaml.
twitter_creds = yaml.safe_load(open('twitter.yaml', 'r').read()) 


# To investigate the Tweets & Sentiment Analysis
# ensure that the latest packages are installed.

nltk.download('stopwords')
nltk.download('vader_lexicon')

In [ ]:
# Pass your Twitter credentials.
twitter_api = Twitter(auth=OAuth(twitter_creds['access_token'],
                                 twitter_creds['access_token_secret'], 
                                 twitter_creds['api_key'],
                                 twitter_creds['api_secret_key'] ))

In [ ]:
# Check Twitter connection.
print(twitter_api)

## Building up the dataset

In [ ]:
# Look for the term cycling on Twitter.
# Ordered by recency.
# Filtering for Tweets only in English.
# Note that the free Twitter API being used here only allows access to the seven most recent
# days of tweets.
# Looks for the most recent 4,000 tweets on cycling each time this is run.

q = {'q':'cycling', 'lang':'en', 'count':100, 'result_type':'recent'}

# Results as an empty list.
results = []

while len(results) < 40:
    query = twitter_api.search.tweets(**q)
    q['max_id'] = query['search_metadata']['next_results'].split('&')[0].split('?max_id=')[1]
    results.append(query)
    
# Determine the number of results.
len(results)

# Repeat this process over several days to get a meaningfully large dataset.
# Continue this once a day for as long as possible.

In [ ]:
# Flatten the results in a DataFrame for NLP Analysis.
df = pd.concat([pd.DataFrame(_['statuses']) for _ in results])

In [ ]:
# Store the data as a backup & in order to build up the data set.
# Each scrape only gathers max 4000 tweets.
# So, to build a large enough unique dataset, have to scrape over several days
df.to_csv('twitter_scrape_191022en.csv')

In [ ]:
# Build up the data source from the daily extracts.
df2 = pd.read_csv('twitter_scrape_071022en.csv')
df3 = pd.read_csv('twitter_scrape_081022en.csv')
df4 = pd.read_csv('twitter_scrape_091022en.csv')
df5 = pd.read_csv('twitter_scrape_101022en.csv')
df6 = pd.read_csv('twitter_scrape_111022en.csv')
df7 = pd.read_csv('twitter_scrape_111022ena.csv')
df8 = pd.read_csv('twitter_scrape_121022en.csv')
df9 = pd.read_csv('twitter_scrape_131022en.csv')
df10 = pd.read_csv('twitter_scrape_131022ena.csv')
df11 = pd.read_csv('twitter_scrape_131022enb.csv')
df12 = pd.read_csv('twitter_scrape_131022enc.csv')
df13 = pd.read_csv('twitter_scrape_141022en.csv')
df14 = pd.read_csv('twitter_scrape_151022en.csv')
df15 = pd.read_csv('twitter_scrape_151022enb.csv')
df16 = pd.read_csv('twitter_scrape_161022en.csv')
df17 = pd.read_csv('twitter_scrape_161022ena.csv')
df18 = pd.read_csv('twitter_scrape_171022en.csv')
df19 = pd.read_csv('twitter_scrape_191022en.csv')

# c.71,240 tweets have been extracted over the past 12 days.
# Provides a meaningfully large dataset for analysis.

In [ ]:
# Creates a master dataset by concatenating the datasets.
Df = pd.concat([df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18, df19])

In [ ]:
# View random samples of the outputs.
Df.sample()

In [ ]:
# Check Metadata.
Df.info()

# 71,244 tweets so far.

In [ ]:
# Parse the created_at column into pd datetime format.
Df['created_at'] = pd.to_datetime(Df.created_at)

In [ ]:
# Search for duplicates & retain just the most recent tweet.
# Look for duplicates and drop them based on created_at and user.

df = Df.drop_duplicates(subset = ['created_at', 'user', 'id'],
  keep = 'last').reset_index(drop = True)

# This avoids duplication of tweets created by the same user, at the same time & same id.
# Duplicates may have come through as part of the scrape.

In [ ]:
# Check Metadata.
df.info()

# Not much geo data captured. 
# User has geo data within it after keyword 'location' per Twitter API documentation.
# 64,661 unique tweets extracted on cycling so far.
# Need to keep running this process every day to build up a meaningfully large database.

In [ ]:
# Extract user details into a seperate dataframe for later analysis.
# Twitter Api docs indicate this field may contain location data.
df1 = df['user'].values

In [ ]:
# View one raw result.
df1[2]

# Confirms user has many variables incl location data.

In [ ]:
# Confirm how the data under user is presently stored.
print(type(df1))

In [ ]:
# Check dimension of numpy array.
np.shape(df1)


In [ ]:
# Convert to pandas dataframe.
user = pd.DataFrame(df1)

In [ ]:
# Double check output.
user.info()
# 1 Column with 64,661 rows of data. 
# Need to transform this where rows become columns.
# user id is stripped of its underlying metadata
# as this is still a small enough file, will do this in Excel.

In [ ]:
# Covert to Excel file for back & re-import later when analysing location data.
user.to_csv('user.csv')


In [ ]:
# Determine the text of the actual tweet
df2 = df['text'].values

In [ ]:
# View one raw result.
df2[9]

# Confirms that the body of the post is stored here

## Prepare the data for NLP & Sentiment Analysis

In [ ]:
# Introduce Stopwords.
stop_words = set(stopwords.words('english'))

In [ ]:
# Split each tweet into individual words.
df2_token = [word_tokenize(_) for _ in df2]

In [ ]:
# Get a list of English words to exclude words that don't appear on the list.
all_english_words = set(words.words())

In [ ]:
# Some pre-processing:
# Get every word.
# Convert it to lowercase.
# Only include if the word is alphanumeric and if it is in the list of English words.

df2_token_nostop =\
[[y.lower() for y in x if y.lower() not in stop_words and y.isalpha() and y.lower() in all_english_words]\
 for x in df2_token]

In [ ]:
# Create a variable to store the Sentiment Intensity Analyser.
darth_vader = SentimentIntensityAnalyzer()

In [ ]:
# Run through a dictionary comprehension to take every cleaned tweet. 
# Next run the polarity score function on the strings.
# This will return four values in a dictionary.

df2_polarity =\
{" ".join(_) : darth_vader.polarity_scores(" ".join(_)) for _ in df2_token_nostop}

In [ ]:
# Convert the list of dictionary results to a pandas dataframe. 
# The index is the cleaned tweet.

polarity_pd = pd.DataFrame(df2_polarity).T

# View the dataframe.
polarity_pd

# Compound score indicates actual sentiment.

## Visualising the data with Charts

### Sentiment

In [ ]:
# Visualise the output in a distribution.
%matplotlib inline
import matplotlib.pyplot as plt

_plot = polarity_pd.reset_index()['compound'].sort_values()
ax1 = plt.axes()
_plot.plot(kind='bar')

x_axis = ax1.axes.get_xaxis()
x_axis.set_visible(False)

plt.show()
plt.close()

- Lots of values are 0 (neutral) - blank spaces.
- More positive sentiment than negative sentiment amongst non-neutral values.
- Some very strong positive sentiment > 0.75.
- Some very strong negative sentiment also visible < -0.75.

A histogram plot will visualise the distribution of sentiment better. The strictly neutral values should be removed to help make the histogram clearer.

In [ ]:
# Remove polarity values equal to zero whch are neutral sentiments.
# This will improve the scale of the histogram and remove all strictly 
# neutral reviews from the analysis.
# This will better highlight the distribution of polarity values = non-neutral sentiment.
# Define the source of the data to use for plotting histogram.
data = polarity_pd['compound'][polarity_pd['compound'] != 0]

In [ ]:
# Visualise the distribution of the sentiment analysis using a histogram.
count, edges, bars = plt.hist(data)
plt.bar_label(bars)
plt.title('Non Neutral Sentiments from most recent Tweets on Cycling')
plt.show()

# The exact numbers will change each time this notebook is rerun as the data will be changed.
# Refresh the analytical content as this changes.

# Many more strongly positive sentiment (>0.75 polarity score) = 1350 tweets.
# Far fewer strongly negative sentiment (<-0.75 polarity score) = 190 tweets.
# Overall, sentiment when non neutral is strongly skewed in favour of cycling.
# Indicates that on the whole from this small sample size, sentiment towards cycling is positive.
# However, recall there are c.64,660 unique tweets.
# Only c.25% of tweets display non 0 sentiment (which are shown in chart below).
# Means vast majority feel very neutral about cycling(shown in chart above).
# Indicates that there is room to improve sentiment towards cycling.
# This can only come from increased cycling uptake and campaign to increase cycling uptake.
# Worthwhile therefore to pursue increasing uptake of cycling.
# Scope to increase cycling uptake established using sentiment analysis given.
# Scope to increase non neutral sentiment on cycling.
# General non neutral sentiment is positive so people should have no objection
# to at least trying out cycling.

### Top Words Visualised in a WordCloud

In [ ]:
# Ensure all words are stored in a list to be used to create a wordcloud.
print(type(df2_token_nostop))

# Confirmed words are in a list.

In [ ]:
# Reformat list into string for use to generate wordcloud.
s = ''.join(str(x) for x in df2_token_nostop)

In [ ]:
# Check the most common items in the list.
# Will identify the obvious words to be removed from the list before wordcloud is generated.
# This will enhance the accuracy of the wordcloud.
from collections import Counter

# split() returns list of all the words in the string.
split_y = s.split()
  
# Pass the split_it list to instance of Counter class.
Counter = Counter(split_y)
  
# most_common() produces k frequently encountered.
# Input values and their respective counts.
most_occur = Counter.most_common(10)
  
print(most_occur)

# From these identify the most common words which fail to convey any meaning or sentiment

In [ ]:
# Remove cycling and other obvious words from the list 
# else they will skew the results.
# Define these words as bespoke stopwords.
stop_words = ['cycling', 'cycling ', ' cycling', ' cycling ', 'distance', 'distance ', 
              'cyclingcycling', 'transport', 'found', 'evidence', 'transport ', 'bike ', 
              'helmet', 'cycle ', 'transport ', 'public', 'society', 'source', 'hour', 'get']

In [ ]:
# Creating new list to avoid messing up orginal list.
new_s = [word for word in s if word not in stop_words]

In [ ]:
# Reformating into string again for wordcloud generation.
st = ''.join(str(x) for x in new_s)

In [ ]:
# Strip all punctuation.
new_string = st.translate(str.maketrans('', '', string.punctuation))

In [ ]:
# Generate Word Cloud using the 30 most frequently used words
# when tweeting about cycling.
wordcloud = (WordCloud(max_words=30, 
                       background_color="white").generate(new_string))

In [ ]:
# Review: Display the wordcloud.
wordcloud.generate(new_string)
plt.figure(figsize=(12, 10))
plt.axis('off') 
plt.imshow(wordcloud, interpolation="lanczos")

# Displays the top 25 most commonly used words by Twitter users when they post about cycling.
# Relative Size displays frequency of the words.
# There are words which suggest an ongoing debate about the potential merit of cycling.
# Words such time, transport, suggest, spend and insane occur within the top 50 words.
# These together, suggest evidence of a public who are still debating the merits of cycling.
# Reinforces the case for the trying to influence cycling uptake as any debate\
# can within reason be influenced with mix of policy and strategic marketing\
# esp on social media. 

## Insights from Geo Data

Objective is to analyse the location of Twitter users posting about cycling. The top most amongst should be the places of most interest to use for comparative analysis against London. 

Further, if London is in the list of places from where people most frequently tweet on cycling, it would show that Londoners are generally animated on the topic and post on the topic on social media. Therefore, it would be worthwhile to try to influence their uptake on cycling through a social media specific strategy.

In [ ]:
# Reintroduce the data on users which contains locational data.
colnames=['user_info']
user = pd.read_csv('user.csv', names=colnames, header=None)

In [ ]:
# Split the user_info column into its constituent parts.
user = user['user_info'].str.split(',', expand=True)

# Drop where index values are NaN.
user = user[user.index.notnull()]

In [ ]:
# View random sample from dataframe.
user.sample()
# Cols 4 and 5 are of interest as they contain location information.
# Col 5 does not always contain location information.
# Has to be included nevertheless to get the data when it does contain location information.

In [ ]:
# Renaming cols of interest.
user.rename(columns = {user.columns[4]: 'city'}, inplace = True)
user.rename(columns = {user.columns[5]: 'country'}, inplace = True)

# Retaining just the cols of interest
user = user[['city', 'country']]


In [ ]:
# Remove all punctuation marks from location columns.
user["city"] = user['city'].str.replace('[^\w\s]','')
user["country"] = user['country'].str.replace('[^\w\s]','')

# Remove all non-alphabet values & cast as string.
user["city"] = user['city'].astype(str).replace('[^a-zA-Z0-9 ]', '', regex=True)
user["country"] = user['country'].astype(str).replace('[^a-zA-Z0-9 ]', '', regex=True)

In [ ]:
# Remove the first two words from the column location.
user['city'] = user['city'].str.split(n=1).str[1]

# Delete all except first word in location 2.
# location 2 appears to sometimes contain additional location value in the first word.
user['country'] =user['country'].drop_duplicates().str.split().str[0]

In [ ]:
# Drop all rows with Null values in location.
# As this is primary search objective, null value rows cannot be interpreted for analysis.
# Removing null value rows will produce cleaner output.
user = user[user.city.notnull()]

In [ ]:
# Fill all Null values in location_2 with Unknown.

user.country = user.country.fillna('Unknown')

# Replace description with unknown in location_2.
# Replace at home with unknown in location.
user = user.replace(['description', 'At home'],['Unknown', 'Unknown'])

In [ ]:
# Count unique values in each column.
for col in user:
  print(col,": ", user[col].nunique())

# >11,000 unique cities
# >1,200 unqiue fields in country

In [ ]:
# Cleaning up the data to ensure mapping is more accurate.
# Aggregating some of the more obvious locations to London\
# incl home counties & general country descriptions.
user['city'] = user['city'].replace(['UK','United Kingdom', 'England', 'NW London', 'N London', 'NE London',
                                    'W London', 'E London', 'S London', 'SE London', 'SW London', 'Berkshire',
                                    'Berks', 'Buckinghamshire', 'Bucks', 'Essex', 'Herts', 'Hertfordshire',
                                    'Kent', 'Surrey', 'Sussex', 'Brixton', 'SE5'],'London')

# Assumes all UK related locations listed above are London unless specified otherwise.

In [ ]:
# Aggregate the city & country counts.
locs = user['city'].value_counts()
locs_2 = user['country'].value_counts()

In [ ]:
# View output from country values.
locs.head(20)

In [ ]:
# Remove Earth from the series as it cannot be assigned to a specific location.
locs=locs.drop('Earth')

In [ ]:
# View output from country values.
locs_2.head(20)

# Less useful as top 3 are clearly unidentifiable locations.
# Before joining this data set with other location data\
# remove the clearly unidentifiable elements.
# Then join the two series to aggregate the counts of the locations.

In [ ]:
# Remove the first 3 elements as they are clearly not identifiable locations.
locs_2 = locs_2.iloc[3:]

In [ ]:
# Stack the 2 series vertically to capture all the location data from both series'.
user_loc = pd.concat([locs_2, locs], axis=0)

# Create a dataframe to analyse the data better.
user_locs = pd.DataFrame({'location':user_loc.index, 'count':user_loc.values})

# Agregate the locational data ahead of visualisation.
user_locs= user_locs.groupby(['location'], as_index=False)['count'].sum()

# Extract the 20 most popular locations by count.
user_locs=user_locs.sort_values(by=['count']).tail(20)

# Ensure datatypes are correctly classified.
user_locs['location'] = user_locs['location'].astype(pd.StringDtype())

# Returned an aggregated list of the top 20 locations from all the tweet user location data.

In [ ]:
# Visualise the top 20 places where people are located when tweeting about cycling
user_locs.plot.barh(x="location", y="count", title="Most Popular locations for tweeting on cycling", 
                   figsize=(8,5), legend=False, fontsize=12, xlabel=' ' )

# People from c.11,000 unique places are tweeting on cycling.
# Can see that out of c.64,000 tweets examined, the most (3, 400+)\
# from a single location are from London.
# c.5% of all people tweeting about cycling globally are readily identifiable as being from London.
# This is very significant share for one city & also vastly outstrips the second most popular location.
# The second most popular & clearly identifiable location is California (c.600 people).

Following feedback from the lo-fi presentation to ThoughtWorks, this area of analysis is being dropped. Whilst insights were generated, feedback from TW was that they werent sufficiently compelling or actionable. As such this area is being dropped although the key highlights are as follows:

- Londoners tweet the most in the world on cycling
- When combined with other locations within the country, UK is the overwhelmingly most popular location 
- WordCloud suggests lack of clarity on the usefulness of cycling
- WordCloud identifed some key topics of interest
- Polarity analysis suggests the existence of a large number of neutrals who have the potential to be influenced

Together, the data suggests that there should be a social media campaign crafted by the client to engage Londoners on the key areas of interest to increase cycling uptake.

# Exploring supplied Data for insights

## Load the Data

In [ ]:
# Load the CSV file(s) for count data only.
ny_counts = pd.read_csv('Bicycle_Counts.csv', parse_dates=['date'], date_parser=d_parser)
inner_london = pd.read_csv('Inner London.csv')
central_london = pd.read_csv('Central London.csv')
outer_london = pd.read_csv('Outer London.csv')
sydney_counts = pd.read_csv('Bicycle_count_surveys.csv')

## Data Wrangling

### Explore the data

In [ ]:
# View the dataframe, metadata, shape.
# Clearly segreate the information for each file.
print("-----------NY Metadata & Info----------->")
ny_counts.info()
print("-----------Inner London Metadata & Info----------->")
inner_london.info()
print("-----------Central London Metadata & Info----------->")
central_london.info()
print("-----------Outer London Metadata & Info----------->")
outer_london.info()
print("-----------Sydney Metadata & Info----------->")
sydney_counts.info()

# New York Dataset
# ny_counts is a large but simple dateframe containing the number of bicycles passing a counters.
# No missing data in ny_counts.  

# London Datasets
# Non-Null values for London dataframes across all columns dont add up. 
# Implies missing data.
# Will need to explore further.
# Central London is another very large datafreame.
# Large dataframes need to be trimmed for unncessary data to reduce strain on memory use.

# Sydney Dataset
# Sydney observations are just aggregated by month and Site ID.
# Sydney observations run for selected hours and not all day (looks like peak hours only).
# Sydney also has no missing values.

In [ ]:
# Get more precise handle on missing values in each dataframe.
inner_london.isnull().sum()

In [ ]:
# Get more precise handle on missing values in each dataframe.
central_london.isnull().sum()

In [ ]:
# Get more precise handle on missing values in each dataframe.
outer_london.isnull().sum()

In [ ]:
# Look at samples of the data.
ny_counts.head(5)

# Very sparse but clean data.
# Id is site Id for where the counter is located.
# Data runs until mid June 2022. 
# So data is very recent.

In [ ]:
# Exploring why NY data is so large.
# View Tail to see end date.
ny_counts.tail(5)

# Data starts from mid Dec 2012.
# Need to explore how far the other data sets run until.

In [ ]:
# Look at samples of the data.
inner_london.sample(5)

# Date has french word in it. Needs to cleaned.

In [ ]:
# Look at samples of the data.
central_london.sample(5)

# Data similar in format to inner london but has some extra columns.
# Will need to trim this dataframe to concatenate.
# Explore whether we need the extra columns and if not will trim.
# Date has french word in it. Needs to cleaned.

In [ ]:
# Look at samples of the data.
outer_london.head(5)

# Matches format of inner london.
# Date has french word in it. Needs to cleaned.

In [ ]:
# Look at samples of the data.
sydney_counts.head(5)

# Very sparse data.
# Counts from just two months per year.
# Counts from just a few hours of the day.

## Filter & Clean Data

In [ ]:
# Rename columns to remove space in column names.
inner_london.columns = inner_london.columns.str.replace(' ','_')
central_london.columns = central_london.columns.str.replace(' ','_')
outer_london.columns = outer_london.columns.str.replace(' ','_')
sydney_counts.columns = sydney_counts.columns.str.replace(' ','_')

In [ ]:
# Remove French Day name in Survey Date. 
inner_london["Survey_date"] = inner_london["Survey_date"].str.replace(r'\D+', '', regex=True)
central_london["Survey_date"] = central_london["Survey_date"].str.replace(r'\D+', '', regex=True)
outer_london["Survey_date"] = outer_london["Survey_date"].str.replace(r'\D+', '', regex=True)

In [ ]:
# Drop empty columns.
ny_counts.dropna(how='all', axis=1, inplace=True)
inner_london.dropna(how='all', axis=1, inplace=True)
central_london.dropna(how='all', axis=1, inplace=True)
outer_london.dropna(how='all', axis=1, inplace=True)

In [ ]:
# Parse London dataframe Dates into appropriate format.

# Convert to datetime format.
inner_london['Survey_date'] = pd.to_datetime(inner_london['Survey_date'], dayfirst=True, yearfirst=False)
central_london['Survey_date'] = pd.to_datetime(central_london['Survey_date'], dayfirst=True, yearfirst=False)
outer_london['Survey_date'] = pd.to_datetime(outer_london['Survey_date'], dayfirst=True, yearfirst=False)

# Sort dataframe in chronological order.
inner_london = inner_london.sort_values(['Survey_date', 'Start_hour', 'Start_minute'])
central_london = central_london.sort_values(['Survey_date', 'Start_hour', 'Start_minute'])
outer_london = outer_london.sort_values(['Survey_date', 'Start_hour', 'Start_minute'])

# Replace with value of previous value row where there is a missing value in Survey_date.
inner_london['Survey_date'].fillna(method='ffill', inplace=True)
central_london['Survey_date'].fillna(method='ffill', inplace=True)
outer_london['Survey_date'].fillna(method='ffill', inplace=True)

# Define day of the week in English and add back as a column.
inner_london['Day_of_week'] = inner_london['Survey_date'].dt.day_name()
central_london['Day_of_week'] = central_london['Survey_date'].dt.day_name()
outer_london['Day_of_week'] = outer_london['Survey_date'].dt.day_name()

# Pass Month into a new column.
inner_london['month'] = inner_london['Survey_date'].dt.month
central_london['month'] = central_london['Survey_date'].dt.month
outer_london['month'] = outer_london['Survey_date'].dt.month

# Pass Year into a new column.
inner_london['year'] = inner_london['Survey_date'].dt.year
central_london['year'] = central_london['Survey_date'].dt.year
outer_london['year'] = outer_london['Survey_date'].dt.year


<font color='red'>
**Change the locale to 'en_GB' for every instance of 'English' if the local machine is Mac**
</font>    

In [ ]:
# Parse London dataframe Dates into appropriate format
# Pass Month name into a new column.
inner_london['month_name'] = inner_london['Survey_date'].dt.month_name(locale='English')
central_london['month_name'] = central_london['Survey_date'].dt.month_name(locale='English')
outer_london['month_name'] = outer_london['Survey_date'].dt.month_name(locale='English')

<font color='red'>
**Change the locale to 'en_GB' for every instance of 'English' if the local machine is Mac**
</font> 

In [ ]:
# Parse NY dataframe Dates into appropriate formats.
# Extract year information from timestamped date column.
ny_counts['year'] = ny_counts['date'].dt.year

# Extract hour information first from timestamped date column.
ny_counts['hour'] = ny_counts['date'].dt.hour

# Extract month name from the timestamped date column.
ny_counts['month_name'] = ny_counts['date'].dt.month_name(locale='en_GB')

# Extract month number from the timestamped date column.
ny_counts['month'] = ny_counts['date'].dt.month

In [ ]:
# Pass Season.
# Will need this to test for seasonal impact on cycling uptake.
# Create function for user defined seasons.
def f(x):
    if (x >= 1) and (x <= 2):
        return 'Winter'
    elif (x > 2) and (x <= 5 ):
        return 'Spring'
    elif (x > 5) and (x <= 8):
        return'Summer'
    elif (x > 8) and (x <= 11) :
        return 'Autumn'
    elif (x > 11):
        return'Winter'

# Apply user defined function to create new column with seasons.
inner_london['season'] = inner_london['month'].apply(f)
central_london['season'] = central_london['month'].apply(f)
outer_london['season'] = outer_london['month'].apply(f)

In [ ]:
# Cleanup NY Dates
# Ensure the data is sorted in chronological order without altering anything else.
ny_counts.sort_values(by='date')

# Data set runs from 31 Aug 2012 to 04 Jul 2022.
# Extract hour of the day to identify patterns. Expect peak vs off peak patterns.
# Group the data by day of the week to look for patterns around that.
# Group the data into month to look for seasonal patterns.
# Group/subset the data by id which is another spatial/location identifier.

In [ ]:
# Create user defined function for peak and off peak hours to match London data.
# See when and if time of the day matters. 
def f(x):
    if (x > 6) and (x <= 7):
        return 'Early Morning'
    elif (x > 7) and (x <= 10 ):
        return 'AM peak'
    elif (x > 10) and (x <= 16):
        return'Inter Peak'
    elif (x > 16) and (x <= 19) :
        return 'PM Peak'
    elif (x > 19) and (x <= 23):
        return'Evening'
    elif (x <= 6):
        return'Night'
    
# Apply user defined function to create new column with peak and off peak hours.
ny_counts['time_of_day'] = ny_counts['hour'].apply(f)

In [ ]:
# Define day of the week and add a column to match London Data.
ny_counts = ny_counts.assign(day_of_week = lambda x: x.date.dt.day_name())

In [ ]:
# Define month to calculate user defined seasons to match London.
# Extract month information first from timestamped date column.
ny_counts['month'] = ny_counts['date'].dt.month

# Create user defined function for seasons.
def f(x):
    if (x >= 1) and (x <= 2):
        return 'Winter'
    elif (x > 2) and (x <= 5 ):
        return 'Spring'
    elif (x > 5) and (x <= 8):
        return'Summer'
    elif (x > 8) and (x <= 11) :
        return 'Autumn'
    elif (x > 11):
        return'Winter'

# Apply user defined function to create new column with seasons.
ny_counts['season'] = ny_counts['month'].apply(f)

In [ ]:
# Reshape the order of the columns so they flow more logically within the dataframe.
neworder = ['id', 'date', 'year', 'month', 'month_name', 'season', 
            'day_of_week', 'hour', 'time_of_day', 'counts', 'status']
ny_counts = ny_counts.reindex(columns=neworder)

In [ ]:
# Check columns of NY_counts.
print(ny_counts.status.unique())

# As both status within acceptable boundaries versus data dictionary, can drop status column

In [ ]:
# Drop redundant columns.
ny_counts=ny_counts.drop(['status'], axis=1)

In [ ]:
# Rename the Sydney columns to align to Peak/off Peak of other dataframes.
sydney_counts.rename(columns = {"Time_0600":"Early_Morning",
                                "Time_0700":"AM_Peak1",
                                "Time_0800":"AM_Peak2",
                                "Time_1600":"PM_Peak1",
                                "Time_1700":"PM_Peak2",
                                "Time_1800":"PM_Peak3"},
                                inplace=True)
 
# Add All AM Peak and PM Peak Columns into 2 columns to be consistent with other dataframes.
sydney_counts['AM_Peak'] = sydney_counts['AM_Peak1'] + sydney_counts['AM_Peak2'] 
sydney_counts['PM_Peak'] = sydney_counts['PM_Peak1'] + sydney_counts['PM_Peak2']\
                         + sydney_counts['PM_Peak3']

# View output with 5 random samples .
sydney_counts.sample(5)

# Can consolidate some columns.

In [ ]:
# Drop rows where total cycles doesnt add up to number of private and hire cycles.
# Create a column to sum the values.
sydney_counts['Sum'] = sydney_counts['AM_Peak'] + sydney_counts['PM_Peak']\
                     + sydney_counts['Early_Morning']

# Drop rows where the column values dont add up.
sydney_counts = sydney_counts[sydney_counts.TotalCount == sydney_counts.Sum]

In [ ]:
# Drop redundant columns.
sydney_counts=sydney_counts.drop(['ObjectId2', 
                                  'AM_Peak1', 
                                  'AM_Peak2', 
                                  'PM_Peak1', 
                                  'PM_Peak2', 
                                  'PM_Peak3', 
                                  'Sum'], axis=1)

In [ ]:
# Drop rows where total cycles doesnt add up to number of private and hire cycles.
# Create a column to sum the values.
inner_london['Sum'] = inner_london['Number_of_private_cycles']\
                    + inner_london['Number_of_cycle_hire_bikes']

# Drop rows where the column values dont add up.
inner_london = inner_london[inner_london.Total_cycles == inner_london.Sum]

In [ ]:
# Drop rows where total cycles doesnt add up to number of private and hire cycles.
# Creating a column to add up the values.
central_london['Sum'] = central_london['Number_of_private_cycles']\
                      + central_london['Number_of_cycle_hire_bikes']

# Drop rows where the column values dont add up.
central_london = central_london[central_london.Total_cycles == central_london.Sum]

In [ ]:
# Drop rows where total cycles doesnt add up to number of private and hire cycles.
# Create a column to add up the values.
outer_london['Sum'] = outer_london['Number_of_male_cycles']\
                    + outer_london['Number_of_female_cycles']\
                    + outer_london['Number_of_unknown_cycles']

# Drop rows where the column values dont add up.
outer_london = outer_london[outer_london.Total_cycles == outer_london.Sum]

In [ ]:
# Drop the decimal point in Survey_wave.
inner_london['Survey_wave_(year)'] = inner_london['Survey_wave_(year)']\
                                    .astype(str).apply(lambda x: x.replace('.0','')).astype(int)

In [ ]:
# Drop redundant columns.
inner_london=inner_london.drop(['Sum', 'Start_hour', 'Start_minute'], axis=1)

In [ ]:
# Drop the additional information in Survey wave column. 
# Now consistent with other London datasets.
central_london["Survey_wave_(calendar_quarter)"] = central_london["Survey_wave_(calendar_quarter)"]\
                                                  .str.replace(r'\D+', '', regex=True)

# Drop the last number in every row.
central_london['Survey_wave_(calendar_quarter)'] = central_london['Survey_wave_(calendar_quarter)']\
                                                  .astype(str).str[:-1].astype(np.int64)

# Rename Column Name to align with other London Datasets
central_london.rename(columns={'Survey_wave_(calendar_quarter)': 'Survey_wave_(year)'},
                                inplace = True)

In [ ]:
# Drop redundant columns.
central_london=central_london.drop(['Sum',
                                    'Start_hour',
                                    'Start_minute',
                                    'Equivalent_financial_quarter'], axis=1)

In [ ]:
# Drop redundant columns.
outer_london=outer_london.drop(['Sum', 
                                'Start_hour',
                                'Start_minute'], axis=1)

In [ ]:
# Count unique values in each column.
for col in inner_london:
  print(col,": ", inner_london[col].nunique())

# More site ids vs location.
# Survey period of over 7 years.
# 5 Periods of day which should be synced in same was as all the other city count data.
# London period of day definition to be used as base.
# 165 weather types need to be consolidated for better analysis.

In [ ]:
# Explore the different types of weather.
print(inner_london.Weather.unique())

# Lots of overlaps for e.g. Rain & Wet, Dry/cold and dry Cold.
# Need to classify into much narrower streams. 
# Cannot meaningfully look for patterns amongst 165 weather conditions!

In [ ]:
# Consolidate descriptions in weather.
# Rain
inner_london['Weather'] = inner_london['Weather'].replace(['Wet','Cloudy/rain','Rain','Mix Wet/dry','Drizzle',
                                                          'Light Showers', 'Mizzle','Windy/rain','Showers',
                                                          'Wet/dry','Wet/damp','Shower','Drizzle/shower','Rainy',
                                                          'wet','Cloudy with showers','Generally overcast brief shower'
                                                          'Light Rain','Shower/dry','Spitting','Drizzle/cloudy',
                                                          'Dry/wet','Damp', 'Dry/drizzle','Dull/damp','Dry-wet',
                                                          'Wet/mix', 'Drizzle/wet','Wet/windy','Rain Shower',
                                                          'Intermittent Showers','Cloudy/drizzle','Rain/drizzle',
                                                          'Wet Road','Drizzle/dry','Drizzle/rain','Mixed Sunny + Rain',
                                                          'Wet/rain', 'V Light Drizzle', 'Rainy', 'W','Slight Drizzle',
                                                          'Rain Stopped', 'Stopped Raining','Wet Rain Stopped','Raining/wet',
                                                          'Showery','Overcast/rain','Rain/wet','Rain/showers','Showers/sunny',
                                                          'Drizzle/showers','Wet/stop Raining','Drizzle Rain','Drizzle Wet',
                                                          'Damp/sun','Raining','Dry + Wet','Showers/cloudy','Cloudy/showers',
                                                          'Getting Wet','Wet Road:sun','Dry But Wet Road','Drizze',
                                                          'wet','Wettish','Light Rain','S.wet','S/w',
                                                          'Heavy Rain','Heavy Shower','Heavy Shr','Down Pour',
                                                           'Deluge','Heavy Showers', 'Shower','Rain Heavy Showers',
                                                           'Intermitent Showers','Thunder Lightening Rain!','Very Wet',
                                                           'V.wet','Heavy Downpour/rain','Showery','Wet Heavy Rain',
                                                           'Wet (heavy Rain)','Wet (shower)'],'Rain')

# Good
inner_london['Weather'] = inner_london['Weather'].replace(['Sunny','Cloudy Sunny','Sun Setting','Good','Dry/sunny',
                                                          'Fine + Dry', 'Fine + Hot','Bright','Dry Hot!!',
                                                          'Dry & Sunny','Dry & Sun','Fine & Dry','Good/dry','Sun',
                                                          'Sunny Dry','Clear and Bright', 'Fine', 'Dry/good', 
                                                          'Fine/dry', 'Warm + Dry','Dry','Dry                         9',
                                                          'Sunny','Cloudy/sunny','Druy','Dry/hot','Dry Warm',
                                                          'Dry/sun','Dryish','Clear And Dry','Clear and Dry','Dry, Warm',
                                                          'Dry, Sunny, Warm','Cloudy with Clear Intervals','Clear and Warm',
                                                          'Dry But Misty','Sunny & Warm All Day','Clear','Dry + Sunny',
                                                          'Sunny/dry','Dr Ry','Dry Y','D','Warm/dry','Bright/dry','Dry Sunny',
                                                          'Fair','Dry/sun','Cloudy','Sunny Overcast Sunny','Sunny/cloudy','Cloudy/rain/sunny',
                                                           'Cloudy + Sunny','Sunny + Cloudy', 'Cloudy/sunny',
                                                           'Bright + Cloudy','Cloudy/dry','Partly Sunny','Dull','Dry & Mild',
                                                           'Cloud','Overcast','Mild','Overcast (No Rain)',
                                                          'Cloudy bright intervals','Generally overcast',
                                                           'Cloudy with clear spells','Sunny Overcast','Dry',
                                                           'Dry/mild', 'Clear','Cloudy and Dry','Partly cloudy but dry',
                                                          'Partly cloudy and dry','Cloudy but dry','Partly cloudy and Dry',
                                                          'Sun/Cloudy','Clouds & Sunny','Sun/clouds','Cloudy & Sunny',
                                                          'Sun & Clouds','Cloudy Dry','Cloud/sun','Mixed','Sun/cloud',
                                                           'Sunny/cloudy','Cloudy Sun','Cloudy/sun','Dry/cloudy',
                                                           'Sun/cloudy','Overcast/dry','Cloud','Dull','Dry/overcast',
                                                          'Dark/cloudy','Cloudy/dry','Cloudy'],'Good')



# Damp
inner_london['Weather'] = inner_london['Weather'].replace(['Wet/dry','Intermittent Light Drizzle','Light Rain',
                                                           'Lt Rain','Drizzle','Intermittent Drizzle', 'Damp','Getting Dry',
                                                           'Dry & Wet','Slight Drizzle/dry','Dry Road Still Wet'],'Damp')

# Dangerous Conditions
inner_london['Weather'] = inner_london['Weather'].replace(['Heavy Rain','Dry/wet Road','Dry With Wet Road',
                                                           'Hot','Snow!','Snow', 'Sleet','Very Hot',
                                                           'Dry (road Wet)','Dry, Sunny, Hot','Very Heavy Rain',
                                                           'Intermittent Heavy Showers','Very Hot/dry','Hot/dry',
                                                           'Storm','Heavy Rain High Winds','V Wet','Rain Heavy',
                                                          'Sunny (hot!)','Heavy Thunder','Overcast/rain Heavy Showers',
                                                          'Too Cold','High Wind','Very Windy','Wet/windy','Wet/v.windy',
                                                           'Wet Hail','Rain/hail','Foggy Wet',
                                                           'Wet Heavy Wind', 'Wet-windy','Hailstones',
                                                           'Short Hail Shower','Rain/sleet','Hail Stone',
                                                          'Hail','Showers/hailstone','Rain/hailstone','Dry Chill','Dry/cold',
                                                           'Dry Cold','Cold/sunny','Cold/cloudy',
                                                           'Dry Very Windy', 'Dry/windy','Windy','Cold','Cloudy/windy',
                                                           'Windy + Sunny','Sunsetting + Windy','Dark Cloudy',
                                                           'Dry V. Cold!','Very Cool','Dry & Windy',
                                                          'Dry but Cold or Wind','Dry/v. Windy','Dry Windy',
                                                          'Windy At First Then Sunny','Windy Dry',
                                                          'Dry Wet Road','Thunder'],'Dangerous_Conditions')

# Consolidating 'Unknown'
inner_london['Weather'] = inner_london['Weather'].replace(['School Out','N/a','Unknown'],'Unknown')

# Transforming Nan Values into Unknown.
# Replacing nan with 'Unknown'.
inner_london.Weather = inner_london.Weather.fillna('Unknown')

# Consolidating "Dry Dark" into "Unknown".
inner_london['Weather'] = inner_london['Weather'].replace(['Dry Dark','Dry/dark','Dark/dry',
                                                           'Dark Dry'],'Unknown')

In [ ]:
# Count unique values in each column.
for col in central_london:
  print(col,": ", central_london[col].nunique())

# More site ids vs location again.
# Survey period of 8 years
# 5 Periods of day which should be synched with all the other city count data.
# Use london period of day definition as base.
# 283 types of weather need to be consolidated.

In [ ]:
# Consolidate descriptions in weather.
# Rain
central_london['Weather'] = central_london['Weather'].replace(['Wet','Cloudy/rain','Rain','Mix Wet/dry','Drizzle',
                                                          'Light Showers', 'Mizzle','Windy/rain','Showers',
                                                          'Wet/dry','Wet/damp','Shower','Drizzle/shower','Rainy',
                                                          'wet','Cloudy with showers','Generally overcast brief shower'
                                                          'Light Rain','Shower/dry','Spitting','Drizzle/cloudy',
                                                          'Dry/wet','Damp', 'Dry/drizzle','Dull/damp','Dry-wet',
                                                          'Wet/mix', 'Drizzle/wet','Wet/windy','Rain Shower',
                                                          'Intermittent Showers','Cloudy/drizzle','Rain/drizzle',
                                                          'Wet Road','Drizzle/dry','Drizzle/rain','Mixed Sunny + Rain',
                                                          'Wet/rain', 'V Light Drizzle', 'Rainy', 'W','Slight Drizzle',
                                                          'Rain Stopped', 'Stopped Raining','Wet Rain Stopped','Raining/wet',
                                                          'Showery','Overcast/rain','Rain/wet','Rain/showers','Showers/sunny',
                                                          'Drizzle/showers','Wet/stop Raining','Drizzle Rain','Drizzle Wet',
                                                          'Damp/sun','Raining','Dry + Wet','Showers/cloudy','Cloudy/showers',
                                                          'Getting Wet','Wet Road:sun','Dry But Wet Road','Drizze',
                                                          'wet','Wettish','Light Rain','S.wet','S/w','Cold/rain',
                                                           'Slightly Wet','Road Wet','Light Shower','Rain Damp','Wet Damp',
                                                              'Wet - Dry','Dry - Wet','Rain Dry','Dry - Rain','Damp - Rain',
                                                              'Wet/ Dry','S. Wet','Cloudy/ Rain','Windy/ Rain','Wet T',
                                                              'Some Showers','Rains','Sunny/rainy','Wetr','Showers Mix',
                                                              'Rain/dry','Rain/cloudy','Shower/wet','Wetter',
                                                              'Heavy Rain','Heavy Shower','Heavy Shr','Down Pour',
                                                           'Deluge','Heavy Showers', 'Shower','Rain Heavy Showers',
                                                           'Intermitent Showers','Thunder Lightening Rain!','Very Wet',
                                                           'V.wet','Heavy Downpour/rain','Showery','Wet Heavy Rain',
                                                           'Wet (heavy Rain)','Wet (shower)','Blustery','V. Wet',
                                                              'Rain & Thunder','Rain-heavy','H Rain','Wert','(rain After)',
                                                              'Cloud/rain','Really Wet','Periods Of Rain Quite Windy',
                                                              'Steady Rain'],'Rain')

# Good
central_london['Weather'] = central_london['Weather'].replace(['Sunny','Cloudy Sunny','Sun Setting','Good','Dry/sunny',
                                                          'Fine + Dry', 'Fine + Hot','Bright','Dry Hot!!',
                                                          'Dry & Sunny','Dry & Sun','Fine & Dry','Good/dry','Sun',
                                                          'Sunny Dry','Clear and Bright', 'Fine', 'Dry/good', 
                                                          'Fine/dry', 'Warm + Dry','Dry','Dry                         9',
                                                          'Sunny','Cloudy/sunny','Druy','Dry/hot','Dry Warm',
                                                          'Dry/sun','Dryish','Clear And Dry','Clear and Dry','Dry, Warm',
                                                          'Dry, Sunny, Warm','Cloudy with Clear Intervals','Clear and Warm',
                                                          'Dry But Misty','Sunny & Warm All Day','Clear','Dry + Sunny',
                                                          'Sunny/dry','Dr Ry','Dry Y','D','Warm/dry','Bright/dry','Dry Sunny',
                                                          'Fair','Dry/sun','Kdry','Fine Windy',
                                                               'Cloudy','Sunny Overcast Sunny','Sunny/cloudy',
                                                               'Cloudy/rain/sunny','Cloudy + Sunny','Sunny + Cloudy',
                                                               'Cloudy/sunny','Bright + Cloudy','Cloudy/dry',
                                                               'Partly Sunny','Dull','Dry & Mild','Cloud','Overcast',
                                                               'Mild','Overcast (No Rain)','Cloudy bright intervals',
                                                               'Generally overcast','Cloudy with clear spells',
                                                               'Sunny Overcast','Dry','Dry/mild', 'Clear',
                                                               'Cloudy and Dry','Partly cloudy but dry',
                                                          'Partly cloudy and dry','Cloudy but dry','Partly cloudy and Dry',
                                                          'Sun/Cloudy','Clouds & Sunny','Sun/clouds','Cloudy & Sunny',
                                                          'Sun & Clouds','Cloudy Dry','Cloud/sun','Mixed','Sun/cloud',
                                                           'Sunny/cloudy','Cloudy Sun','Cloudy/sun','Dry/cloudy',
                                                           'Sun/cloudy','Overcast/dry','Cloud','Dull','Dry/overcast',
                                                          'Dark/cloudy','Cloudy/dry','Cloudy','Hazy','Partly Cloudy',
                                                               'Drty','Dry (windy)','Fine (windy)','Sunny Cloudy',
                                                              'Dry Dark','Dark','Dry Mon','Dry Wed','Dry Thu','Dry Fri',
                                                              'Sun/rain','Thunder','Cloudy','Sunny Overcast Sunny',
                                                               'Sunny/cloudy','Cloudy/rain/sunny',
                                                           'Cloudy + Sunny','Sunny + Cloudy', 'Cloudy/sunny',
                                                           'Bright + Cloudy','Cloudy/dry','Partly Sunny','Dull','Dry & Mild',
                                                           'Cloud','Overcast','Mild','Overcast (No Rain)',
                                                          'Cloudy bright intervals','Generally overcast',
                                                           'Cloudy with clear spells','Sunny Overcast','Dry',
                                                           'Dry/mild', 'Clear','Cloudy and Dry','Partly cloudy but dry',
                                                          'Partly cloudy and dry','Cloudy but dry','Partly cloudy and Dry',
                                                          'Sun/Cloudy','Clouds & Sunny','Sun/clouds','Cloudy & Sunny',
                                                          'Sun & Clouds','Cloudy Dry','Cloud/sun','Mixed','Sun/cloud',
                                                           'Sunny/cloudy','Cloudy Sun','Cloudy/sun','Dry/cloudy',
                                                           'Sun/cloudy','Overcast/dry','Cloud','Dull','Dry/overcast',
                                                          'Dark/cloudy','Cloudy/dry','Cloudy','Hazy','Partly Cloudy',
                                                               'Drty','Dry (windy)','Fine (windy)','Sunny Cloudy',
                                                              'Dry Dark','Dark','Dry Mon','Dry Wed','Dry Thu','Dry Fri',
                                                              'Sun/rain','Thunder','Ddry','Dy','Dry/sunny/cold','Fine Cold',
                                                              'Cold Dry','Dry & Cold','Dry And Fine','Dry And Sunny',
                                                              'Dry And Warm','Fine And Dry','Warm + Sunny','Warm And Humid',
                                                              'Warm And Windy','Overcast And Dull','Cloudy And Warm',
                                                              'Sunny Periods And Warm','Dry And Windy','Dry And Very Windy',
                                                              'Warm Sunny And Windy','Hot And Humid','Mild And Sunny',
                                                               'Warm And Overcast','Sunny & Windy','Windy/cloudy',
                                                              'Dry/gusty','Coldish','Windy/dry','Dry But A Bit Windy',
                                                               'Sunny Cold','Cold At First Then Warm/sunny',
                                                              'Warm & Sunny Chilly Later','Fine + Dry Chilly At First',
                                                               'Fine & Sunny','dry','A Bit Chilly At First',
                                                               'Warm With A Slight Wind','Cold Then Dry And Windy',
                                                               'Dry And Overcast','Warm + Sunny Cloudy + Windy',
                                                              'Dry 3/4 Dry','Sunny Until Evening But Windy',
                                                               'Winds Rather Chilly','Warm','Sunny But Very Windy',
                                                               'Now Starts To Get Chilly'],'Good')


# Light Rain
central_london['Weather'] = central_london['Weather'].replace(['Wet/dry','Intermittent Light Drizzle','Light Rain',
                                                           'Lt Rain','Drizzle','Intermittent Drizzle', 'Damp','Getting Dry',
                                                           'Dry & Wet','Slight Drizzle/dry','Wet Intermittently',
                                                               'Light Rain','V Light Rain','Dry Wet Road','Dry A.m Wet P.m',
                                                               'Mist','Road Drying Sun Out','Wetish','Light Shrs',
                                                              'Fine Drizzle','V Light Shrs','L/rain','Rain Stopped-dry',
                                                              'V Lt Rain','V.light Rain','Dry (+brief Speels Of Drizzle',
                                                              'Wet (spitting)','Drizzly Rain','Almost Dry','Damp & Drizzly',
                                                              'Dry Road Wet With Leaves','Wet Drizzle','No Rain Wet Roads',
                                                              'Dry But Wet Roads','Very Light Rain','Light Drizzle',
                                                              'Dry/wet Road Surface','V Light Showers','V. Light Rain',
                                                              'Wet/cloudy','Wet/sunny','Dry Road Still Wet',
                                                              '2 Snowflakes Otherwise Dry','Wet-dry','Dry/drizzly',
                                                              'Wet/light Showers','Wet/drizzle','Wet And Windy',
                                                              'Drizzling','Drizzle Damp','Windy Showery','Wet + Dry',
                                                              'V.light Drizzle','Very Light Drizzle','Drying Up','Wet Again',
                                                              'Cold Sunny Rain','Wet First Then Dry','Wetr First Then Dry',
                                                              'Dry With Intermitent Rain','(drizzle)','Damp/misty/wet',
                                                              'Dry But Rain Threatening','Slight Drizzle Till End',
                                                              'Damp/misty','Cold & Dry Early Rain Later',
                                                              'Wet ','Windy/drizzle','Intermitent Light Showers',
                                                              'Intermitent Light Rain','A Few Rain Showers','Drizzly',
                                                              'Rain Looking Likely','A Few Drops Of Rain'],'Damp')

# Dangerous Weather
central_london['Weather'] = central_london['Weather'].replace(['Heavy Rain','Dry/wet Road','Dry With Wet Road',
                                                           'Hot','Snow!','Snow', 'Sleet','Very Hot',
                                                           'Dry (road Wet)','Dry, Sunny, Hot','Very Heavy Rain',
                                                           'Intermittent Heavy Showers','Very Hot/dry','Hot/dry',
                                                           'Storm','Heavy Rain High Winds','V Wet','Rain Heavy',
                                                          'Sunny (hot!)','Heavy Thunder','Overcast/rain Heavy Showers',
                                                          'Too Cold','High Wind','Very Windy','Dry & Very Windy',
                                                              'Very Hot Dry','Wet/windy','Wet/v.windy','Wet Hail',
                                                               'Rain/hail','Foggy Wet',
                                                           'Wet Heavy Wind', 'Wet-windy','Hailstones',
                                                           'Short Hail Shower','Rain/sleet','Hail Stone',
                                                          'Hail','Showers/hailstone','Rain/hailstone','Cold/ Rain',
                                                              'Foggy','Wet & Windy','Wet + Windy','Rain/wind',
                                                              'Wet (windy)','Occasional Lt Snow Shrs',
                                                              'Wet And Very Windy','Dry Chill','Dry/cold','Dry Cold',
                                                               'Cold/sunny','Cold/cloudy',
                                                           'Dry Very Windy', 'Dry/windy','Windy','Cold','Cloudy/windy',
                                                           'Windy + Sunny','Sunsetting + Windy','Dark Cloudy',
                                                           'Dry V. Cold!','Very Cool','Dry & Windy',
                                                          'Dry but Cold or Wind','Dry/v. Windy','Dry Windy',
                                                          'Windy At First Then Sunny','Windy Dry','Cold Windy Dry',
                                                              'Cold/dry','Some Heavy Showers','Very Cold/dry',
                                                              'Foggy/v Cold','Hail Shower','Snowing','Wet/ Snowing',
                                                              'Heavy Snow','Dry/very Windy','Very Windy & Cold',
                                                              'Wet Light Hailstone','Heavy Showers Throughout Day',
                                                              'High Winds & Spits Of Rain','Fine V Cold',
                                                              'Dry (frost & Fog)','V Cold Showers','Cold/showery',
                                                              'Light Showers Inc Some Hail','Cloudy/hail','Cold Wind',
                                                              'Hot & Sunny','Hot And Sunny','Dry/windy/strong Wind',
                                                              'Hot + Humid','Very Cold Sunny But Windy'],'Dangerous_Conditions')

# Consolidating 'Unknown'
central_london['Weather'] = central_london['Weather'].replace(['School Out','N/a','Unknown','Dark Sunny',
                                                              'Wed','Warm & Sunny But Windy & Cold'],'Unknown')

# Transforming Nan Values into Unknown
# Replacing nan with 'Unknown'
central_london.Weather = central_london.Weather.fillna('Unknown')

# Consolidating "Dry Dark"
central_london['Weather'] = central_london['Weather'].replace(['Dry Dark','Dry/dark','Dark/dry',
                                                           'Dark Dry', 'X'],'Unknown')

In [ ]:
# Count unique values in each column.
for col in outer_london:
  print(col,": ", outer_london[col].nunique())

# More site ids vs location.
# May imply multiple sites in same location. 
# Survey period of over 7 years.
# 5 Periods of day which should be synched in same fashion with all the other city count data.
# Use london period of day definition as base.
# 124 types of weather needs to be consolidated

In [ ]:
# Consolidate descriptions in weather.

# Rain.
outer_london['Weather'] = outer_london['Weather'].replace(['Wet','Showers','Rain','Cloudy + Rain','Rain & Cloudy',
                                                          'Raining', 'Rain/cloudy','Wet/thunder','Light Showers',
                                                          'Rain/showers','W','Wey','Drizzle/shower','Rainy',
                                                          'wet','Cloudy with showers','Generally overcast brief shower',
                                                          'Heavy Rain','Heavy Shower','Heavy Shr','Down Pour',
                                                           'Deluge','Heavy Showers', 'Shower','Rain Heavy Showers',
                                                           'Intermitent Showers','Thunder Lightening Rain!','Very Wet',
                                                           'V.wet','Heavy Downpour/rain','Showery','Wet Heavy Rain',
                                                           'Wet (heavy Rain)'],'Rain')

# Good.
outer_london['Weather'] = outer_london['Weather'].replace(['Cloudy','Sunny Overcast Sunny','Sunny/cloudy','Cloudy/rain/sunny',
                                                           'Cloudy + Sunny','Sunny + Cloudy', 'Cloudy/sunny',
                                                           'Bright + Cloudy','Cloudy/dry','Partly Sunny','Dull','Dry & Mild',
                                                           'Cloud','Overcast','Mild','Overcast (No Rain)',
                                                          'Cloudy bright intervals','Generally overcast',
                                                           'Cloudy with clear spells','Sunny Overcast','Dry',
                                                           'Dry/mild', 'Clear','Cloudy with clear spells',
                                                          'Sunny Overcast', 'Sunny','Cloudy Sunny','Sun Setting','Good',
                                                           'Dry/sunny','Fine + Dry', 'Fine + Hot','Bright','Dry Hot!!',
                                                          'Dry & Sunny','Dry & Sun','Fine & Dry','Good/dry','Sun',
                                                          'Sunny Dry','Clear and Bright', 'Fine', 'Dry/good', 
                                                          'Fine/dry','Warm + Dry','D'],'Good')
# Damp.
outer_london['Weather'] = outer_london['Weather'].replace(['Wet/dry','Intermittent Light Drizzle',
                                                           'Light Rain','Lt Rain','Drizzle','Intermittent Drizzle', 'Damp',
                                                           'Getting Dry','Dry & Wet','Dry/wet',],'Damp')


# Dangerous conditions.
outer_london['Weather'] = outer_london['Weather'].replace(['Dry Chill','Dry/cold','Dry Cold','Cold/sunny','Cold/cloudy',
                                                           'Dry Very Windy', 'Dry/windy','Windy','Cold','Cloudy/windy',
                                                           'Windy + Sunny','Sunsetting + Windy','Dark Cloudy',
                                                           'Dry V. Cold!','Very Cool','Wet/windy','Wet/v.windy','Wet Hail',
                                                           'Rain/hail','Foggy Wet','Wet Heavy Wind', 'Wet-windy','Hailstones',
                                                           'Short Hail Shower','Heavy Rain','Dry/wet Road','Dry With Wet Road',
                                                           'Hot','Snow!','Snow', 'Sleet','Very Hot','Dry (road Wet)'],
                                                          'Dangerous_Conditions')

# Replacing nan with 'Unknown'.
outer_london.Weather = outer_london.Weather.fillna('Unknown')

# Consolidating 'Unknown'.
outer_london['Weather'] = outer_london['Weather'].replace(['Dry Dark','Dry/dark','Dark/dry','Dark Dry', 'N/a'],'Unknown')

In [ ]:
# Check for duplicated values in the dataframes.
# Duplicates are rows where values across all columns match with another row in the dataframe.
duplicateRowsDF_inner_london = inner_london[inner_london.duplicated()]

In [ ]:
# View output.
print(duplicateRowsDF_inner_london)

# 17,862 rows of duplicates.
# Will drop once all data cleaning is done.

In [ ]:
# Check for duplicated values in the dataframes.
duplicateRowsDF_central_london = central_london[central_london.duplicated()]

In [ ]:
# View output.
print(duplicateRowsDF_central_london)

# 431 rows of duplicates.
# Will drop these but once all data cleaning is done.

In [ ]:
# Check for duplicated values in the dataframes.
# Duplicates are rows where values across all columns match with another row in the dataframe.
duplicateRowsDF_outer_london = outer_london[outer_london.duplicated()]

In [ ]:
# View output.
print(duplicateRowsDF_outer_london)

# No duplicates.

In [ ]:
# Check for duplicated rows across all column values
duplicateRowsDF_ny = ny_counts[ny_counts.duplicated()]

In [ ]:
# See output of duplication check.
print(duplicateRowsDF_ny)

# No duplicates.

In [ ]:
# Check for duplicated rows across all column values.
duplicateRowsDF_sydney = sydney_counts[sydney_counts.duplicated()]

In [ ]:
# See output of duplication check.
print(duplicateRowsDF_sydney)

# No duplicates.

### Summarising Null Values

In [ ]:
# Check missing values.
ny_counts.isnull().sum()

# No Null Values.

In [ ]:
# Check missing values.
sydney_counts.isnull().sum()

# No Null Values

In [ ]:
# Check missing values.
inner_london.isnull().sum()

# Just 6 null values in Time. 
# Given the overall size of the dataframe, this is not worthwhile trying to investigate.
# Will drop them.

In [ ]:
# Check missing values.
central_london.isnull().sum()

# No Null Values.

In [ ]:
# Check missing values.
outer_london.isnull().sum()

# No Null Values.

## Exploring Distribution of count data

### New York

In [ ]:
# Group NY Counts Data by year, month, day_of week & time of day.
df = ny_counts.groupby(['year'], as_index=False)['counts'].count()
df1= ny_counts.groupby(['month_name', 'month'], as_index=False)['counts'].count()
df2 = ny_counts.groupby(['day_of_week'], as_index=False)['counts'].count()
df3 = ny_counts.groupby(['time_of_day', 'hour'], as_index=False)['counts'].count()
df4 = ny_counts.groupby(['year', 'month'], as_index=False)['counts'].count()

In [ ]:
# Sort values.
df1.sort_values('month', inplace=True)
df3.sort_values('hour', inplace=True)

In [ ]:
# Visualise NY Counts data distribution by year.
sns.set_theme(style="whitegrid")
from matplotlib.ticker import FormatStrFormatter

# Initialise matplotlib fiture.
f, g = plt.subplots(figsize = (10,5))

g=sns.barplot(x = df['year'], y = df['counts'])

# Add legend and add title etc.
g.set(ylabel = "Number of counts",
     xlabel = "Year")
g.set_title('Distribution of count data by year in New York')
sns.despine(left=True, bottom=True)

# Show the plot.
plt.show()

# We dont have equal amounts of count data across the years.
# Suggests lack of uniformity across the frequency distribution of the count data.

In [ ]:
# Visualise NY Counts count data distribution by month.
sns.set_theme(style="whitegrid")
from matplotlib.ticker import FormatStrFormatter

# Initialise matplotlib fiture.
f, g = plt.subplots(figsize = (10,5))

g=sns.barplot(x = df1['month_name'], y = df1['counts'])

# Add legend and add title etc.
g.set(ylabel = "Number of counts",
     xlabel = "Month")
g.set_title('Distribution of count data by month in New York')
sns.despine(left=True, bottom=True)

# Show the plot.
plt.show()

# Uneven distribution confirmed.

In [ ]:
# Visualise NY Counts data distribution by day of the week.
sns.set_theme(style="whitegrid")
from matplotlib.ticker import FormatStrFormatter

# Initialise matplotlib fiture.
f, g = plt.subplots(figsize = (10,5))

g=sns.barplot(x = df2['day_of_week'], y = df2['counts'])

# Add legend and add title etc.
g.set(ylabel = "Number of counts",
     xlabel = "Day of the week")
g.set_title('Distribution of count data by day of the week in New York')
sns.despine(left=True, bottom=True)

# Show the plot.
plt.show()

# Equal.

In [ ]:
# Visualise Ny Counts by time of day.
sns.set_theme(style="whitegrid")
from matplotlib.ticker import FormatStrFormatter

# Initialise matplotlib figure.
f, g = plt.subplots(figsize = (10,5))

g=sns.barplot(x = df3['time_of_day'], y = df3['counts'])

# Add legend and add title etc.
g.set(ylabel = "Number of counts",
     xlabel = "Time of the day")
g.set_title('Distribution of count data by time of the day in New York')
sns.despine(left=True, bottom=True)

# Show the plot.
plt.show()

# Equal.

### London

Inner London

In [ ]:
# Group the data.
df = inner_london.groupby(['Survey_date'], as_index=False)['Total_cycles'].count()

In [ ]:
# Add a count rank column.
# Define a private function to fill the count rank column with relative numbers.
# If Total cycles is > 0 then fill with 1 to 10 in the value column.
# Else 0.

# function for assigning value to Total cycle count of the day for sorting.
def f(x):
    if (x < 1):
        return '0'
    if (x >= 1) and (x <= 100):
        return '1'
    elif (x > 100) and (x <= 300 ):
        return '2.5'
    elif (x > 300) and (x <= 600):
        return'4.5'
    elif (x > 600) and (x <= 1000) :
        return '8'
    elif (x > 1000):
        return'10'
     

# Apply user defined function to create new column with seasons.
df['count_rank'] = df['Total_cycles'].apply(f) 

In [ ]:
# Ensure Count Rank is a numeric value
df['count_rank'] = pd.to_numeric(df['count_rank'])

In [ ]:
# Plot a Calendar Heatmap to display relative number of counts for each day in a calendar view
# Create the plot
fig = calplot(df, 
              x="Survey_date", 
              y="count_rank", 
              dark_theme=True, 
              gap=0,
              years_title=True)

fig.show()

# Darker the block, the more data there is for that day.
# Lighter the block means less data for that day.
# Black area means no data for that day.
# Shows Data Distribution for inner London is very patchy.
# Concentrated just between April and July most years.
# Not evenly distributed even in that limited period.
# No weekend counts.
# No count in the usual summer months in 2020.
# Instead counts in autumn months in 2020.
# Normal counting activity returns in 2021.

Outer London

In [ ]:
# Group the data.
df = outer_london.groupby(['Survey_date'], 
                          as_index=False)['Total_cycles'].count()

In [ ]:
# Add a count rank column.
# Define a private function to fill the count rank column with relative numbers.
# If Total cycles is > 0 then fill with 1 to 10 in the value column.
# Else 0.

# Function to assig a value to Total cycle count of the day for sorting.
def f(x):
    if (x < 1):
        return '0'
    if (x >= 1) and (x <= 100):
        return '1'
    elif (x > 100) and (x <= 300 ):
        return '2.5'
    elif (x > 300) and (x <= 600):
        return'4.5'
    elif (x > 600) and (x <= 1000) :
        return '8'
    elif (x > 1000):
        return'10'
     

# Apply user defined function to create new column with seasons.
df['count_rank'] = df['Total_cycles'].apply(f) 

In [ ]:
# Ensure Count Rank is a numeric value.
df['count_rank'] = pd.to_numeric(df['count_rank'])

In [ ]:
# Plot a Calendar Heatmap to display relative number of counts for each day in a calendar view.
# create the plot; same logic as Inner London
fig = calplot(df, 
              x="Survey_date", 
              y="count_rank", 
              dark_theme=True, 
              gap=0, 
              colorscale = "purples", 
              years_title=True)
fig.show()

# Very similar patterns to Inner London.
# Data Very patchy and not collected continously through the year.
# Black areas are where there is 0 count.
# As counts increase on a given day, blocks get darker purple.

Central London

In [ ]:
central_london.head(3)

In [ ]:
# Group the data.
df = central_london.groupby(['year'], as_index=False)['Total_cycles'].count()
df1= central_london.groupby(['month_name', 'month'], as_index=False)['Total_cycles'].count()
df2 = central_london.groupby(['Day_of_week'], as_index=False)['Total_cycles'].count()
df3 = central_london.groupby(['Period', 'Time'], as_index=False)['Total_cycles'].count()
df4 = central_london.groupby(['Survey_date'], as_index=False)['Total_cycles'].count()

In [ ]:
# Sort values to improve chart presentation.
df1.sort_values('month', inplace=True)
df3.sort_values('Time', inplace=True)

In [ ]:
# Visualise Central London counts data distribution by year.
sns.set_theme(style="whitegrid")
from matplotlib.ticker import FormatStrFormatter

# Initialise matplotlib figure.
f, g = plt.subplots(figsize = (10,5))

g=sns.barplot(x = df['year'], y = df['Total_cycles'])

# Add legend and add title etc.
g.set(ylabel = "Number of counts",
      xlabel = "Year")

g.set_title('Distribution of count data by year in Central London')
sns.despine(left=True, bottom=True)

# Show the plot.
plt.show()

# Reasonably equal distribution.
# Except 2020 which again matches Inner London where there is a dip in number of counts too.
# Therefore during Covid this count service was impacted.
# Data Collection should be automated.

In [ ]:
# Visualise Central London count data distribution by month.
sns.set_theme(style="whitegrid")
from matplotlib.ticker import FormatStrFormatter

# Initialise matplotlib figure.
f, g = plt.subplots(figsize = (12,5))

g=sns.barplot(x = df1['month_name'], y = df1['Total_cycles'])

# Add legend and add title etc.
g.set(ylabel = "Number of counts",
     xlabel = "Month")
g.set_title('Distribution of count data by month in Central London')
sns.despine(left=True, bottom=True)

# Show the plot.
plt.show()

# Very Unequal distribution.
# Data missing without any clear pattern.
# But data is there across all months.

In [ ]:
# Visualise Central London counts data distribution by day of the week.
sns.set_theme(style="whitegrid")
from matplotlib.ticker import FormatStrFormatter

# Initialise matplotlib figure.
f, g = plt.subplots(figsize = (10,5))

g=sns.barplot(x = df2['Day_of_week'], y = df2['Total_cycles'])

# Add legend and add title etc.
g.set(ylabel = "Number of counts",
      xlabel = "Day of the week")
g.set_title('Distribution of count data by day of the week in Central London')
sns.despine(left=True, bottom=True)

# Show the plot.
plt.show()

# Very uneven distribution although counts present every day.
# Weekends clearly very uneven.
# Given they are on the chart implies there is some data for weekends.
# But too few to appear reasonably within the scale of the other data.
# Again similar distribution to Inner & Outer London here.

In [ ]:
# Visualise Central London by time of day.
sns.set_theme(style="whitegrid")
from matplotlib.ticker import FormatStrFormatter

# Initialise matplotlib figure.
f, g = plt.subplots(figsize = (12,5))

g=sns.barplot(x = df3['Period'], y = df3['Total_cycles'])

# Add legend and add title etc.
g.set(ylabel = "Number of counts",
     xlabel = "Time of the day")
g.set_title('Distribution of count data by time of the day in Central London')
sns.despine(left=True, bottom=True)

# Show the plot.
plt.show()

# Equal.

In [ ]:
# Add a count rank column.
# Define a private function to fill the count rank column with relative numbers.
# If Total cycles is > 0 then fill with 1 to 10 in the value column.
# Else 0.

# function for assigning value to Total cycle count of the day for sorting.
def f(x):
    if (x < 1):
        return '0'
    if (x >= 1) and (x <= 300):
        return '1'
    elif (x > 300) and (x <= 600 ):
        return '2'
    elif (x > 600) and (x <= 800):
        return'3'
    elif (x > 800) and (x <= 900) :
        return '4'
    elif (x > 900) and (x <= 1000):
        return'5'
    elif (x > 1000) and (x <= 1200) :
        return '6'
    elif (x > 1200) and (x <= 1500):
        return'7'
    elif (x > 1500) and (x <= 2000) :
        return '8'
    elif (x > 2000) and (x <= 3000):
        return'9'
    elif (x > 3000):
        return'10'
     

# Apply user defined function to create new column with seasons.
df4['count_rank'] = df4['Total_cycles'].apply(f) 

In [ ]:
# Ensure Count Rank is a numeric value.
df4['count_rank'] = pd.to_numeric(df4['count_rank'])

In [ ]:
# Plot a Calendar Heatmap to display relative number of counts for each day in a calendar view.
# Create the plot; same logic as previous calendar heatmaps.
fig = calplot(df4, x="Survey_date", y="count_rank", dark_theme=True, gap=0, 
              colorscale = "blues", years_title=True)
fig.show()

# Black areas which show no count data registered at all for those days.
# Rest of the count data very patchy and incomplete too as displayed by the relative heatmap.
# Counting stopped altogether in the 3 phases of lockdowns during Covid.
# No weekend data similar to the other London data sets.

Sydney

In [ ]:
# Group the data.
df = sydney_counts.groupby(['Year'], as_index=False)['TotalCount'].count()
df1= sydney_counts.groupby(['Month'], as_index=False)['TotalCount'].count()
df2 = sydney_counts.groupby(['SiteID'], as_index=False)['TotalCount'].count()

In [ ]:
# Sort values to improve chart presentation.
df.sort_values('Year', inplace=True)
df1.sort_values('Month', inplace=True)

In [ ]:
# Visualise Sydney counts data distribution by year.
sns.set_theme(style="whitegrid")
from matplotlib.ticker import FormatStrFormatter

# Initialise matplotlib figure.
f, g = plt.subplots(figsize = (10,5))

g=sns.barplot(x = df['Year'], y = df['TotalCount'])

# Add legend and add title etc.
g.set(ylabel = "Number of counts",
     xlabel = "Year")
g.set_title('Distribution of count data by year in Sydney')
sns.despine(left=True, bottom=True)

# Show the plot.
plt.show()

# Sydney very consistent until 2017.
# Sudden drop in count of counts after that.
# Pick up of count data in Sydnet again 2019 
# Again pretty uniform albeit consistenly fewer counts compared to pre-2018.
# Was there a policy shift in 2018 w.r.t. cycling & thus resources pointed towards counting?

In [ ]:
# Visualise Sydney count data distribution by month.
sns.set_theme(style="whitegrid")
from matplotlib.ticker import FormatStrFormatter

# Initialise matplotlib figure
f, g = plt.subplots(figsize = (10,5))

g=sns.barplot(x = df1['Month'], y = df1['TotalCount'])

# Add legend and add title etc.
g.set(ylabel = "Number of counts",
     xlabel = "Month")
g.set_title('Distribution of count data by month in Sydney')
sns.despine(left=True, bottom=True)

# Show the plot.
plt.show()

# Equally distributed.

## Data Imputation or Deletion

New York & London have lots of patchy missingness.
Further gap is missing weekend data in London.
Strip New York of all weekend data which will then allow direct comparison between the cities.

Missing data on weekends or some months only means MAR
Some of the unevenness of data is also MCAR such as unevenesss between months in New York
Most of the missing data in London is MAR
 
New York data missingness is simplest as the data appears to be missing completely at random (MCAR)
There is no observable link between missing data and the independent variables
In order to test this statistically the categorical data needs to be converted 
Following the lo-fi presentation to ThoughtWorks, repairing data missingness was dropped
It was deemed to be not worth the effort on balance

In [ ]:
# Drop all rows in New York Count where day of the week is saturday or sunday.
# This will align to fact London has no weekend data.
# Generates a more like for like comparison between New York and London.
# Drop rows containing Saturday or Sunday in day_of_week.
ny_countsA = ny_counts[ny_counts["day_of_week"].str.contains("Saturday|Sunday") == False]

In [ ]:
# Saving the DataFrames to csv.
# Save the cleaned dataframes in NY and Sydney.
# Additionally save the NY dataframe without the weekend day seperately.
ny_counts.to_csv('ny_counts_cleaned_201022.csv')
ny_countsA.to_csv('ny_counts_weekdays_only_201022.csv')
sydney_counts.to_csv('sydney_counts_cleaned_201022.csv')

# Data now is cleaned.
# Data will be saved.
# No outlier detection carried.
# Data is assumed to be ok but again we should test for this ahead of final presentation.
# If further analysis doesnt obviously make sense will return for outlier detection.

In [ ]:
# Load the dataframe for NY without weekend.
ny_counts = pd.read_csv('ny_counts_weekdays_only_201022.csv')

## Merging DataFrames

### Concatenate London DataFrames

In [ ]:
# Merging London Dataframes first to check output as this is the base Df. 
london = pd.concat([outer_london, central_london, inner_london])

In [ ]:
# View Metadata.
london.info()

# London rows all add up correctly so merge is successful. 
# Expecting some null values now as inner and central london dfs didnt have gender info.
# Expecting some null values now as outer london df didnt have cycle ownership/rental info.

In [ ]:
# Check missing values
london.isnull().sum()

# All aligns as expected. 
# Will drop the 6 rows where period and time has null values.
# 6 rows being deleted will not impact data with 1.287mio rows\
# and doesnt merit time to investigate these.
# Gender & Cycle ownership data was already missing from some of the original data.
# The missing rows add up to the sum of the original data where the data was missing\
# so no new duplicates generated.
# Confirms concatenation is successful.

In [ ]:
# Dropping the 6 rows where missing values in Time column.
london = london.dropna(subset=['Time'])

In [ ]:
# Merging the Count Data from london with spatial data.
# Pull in the Spatial data.
# This is a new file where the spatial information from TFL has been coverted.
# "Northing" & "Easting" converted into more universally conventional "Longitude" & "Latitude".
# Reading the new file as London_Biking_sites_reconv.xlsx which is supplied as additional file.
bike_site = pd.read_excel("London_Biking_sites_reconv.xlsx")

# Contains additional data. 
# The base TFL supplied data's spatial information has been converted into Longtitude & Latitude.

In [ ]:
# Look at Metadata.
bike_site.info()

In [ ]:
# Check for duplicates in bike sites info.
duplicateRowsDF_bike_sites = bike_site[bike_site.duplicated()]

In [ ]:
# View output from duplicate check.
print(duplicateRowsDF_bike_sites)

# No duplicate values in bike sites info.

In [ ]:
# Check missing values.
bike_site.isnull().sum()

# Two rows with missing values.
# Again give size of dataset these can be dropped without further investigation.

In [ ]:
# View Data Sample.
bike_site.sample(5)

In [ ]:
# Renaming Column on spatial data to align with Count Data.
bike_site.rename(columns = {"UnqID": "Site_ID"},inplace=True)

In [ ]:
# Rename columns to remove space in column names.
bike_site.columns = bike_site.columns.str.replace(' ','_')

In [ ]:
# Dropping the 2 missing value rows previously identified.
bike_site = bike_site.dropna(subset=['Functional_cycling_area'])

In [ ]:
# Merge the dataframes.
dfs = [bike_site, london]
london_complete = ft.reduce(lambda left, right: pd.merge(left, right, on='Site_ID'), dfs)

In [ ]:
# Check for duplicates in merged Df. 
# Merged Df created using new library.
duplicateRowsDF_london_complete = london_complete[london_complete.duplicated()]

In [ ]:
# View Metadata of combined df.
london_complete.info()

In [ ]:
# Check if Site ID & Prog ID are identical on string test.
# Strip integers from SiteID into a column containing just the string values.
london_complete['Id_Check']= london_complete['Site_ID'].str.replace('\d+', '')

In [ ]:
# Check the stripped strings for match with ProgID.
# If no match, output into new column will be FALSE.
london_complete['Equality_Test'] = london_complete['Id_Check'].equals(london_complete['ProgID'])

# Isolating the different unique values in this column.
uniqueValues = london_complete['Equality_Test'].nunique()

# Displaying the number of  unique values.
print(london_complete["Equality_Test"].unique())

# Established that ProgID (which contains strings only) is duplicate of Site_ID.
# ProgID can be dropped.

In [ ]:
# Remove overlapping spatial data.
london_complete = london_complete.drop(['ProgID', 
                                        'Easting', 
                                        'Northing', 
                                        'Location_y', 
                                        'Id_Check',
                                        'Equality_Test'], axis=1)

In [ ]:
# Change all column names to lower case to ease recalling them for analysis.
london_complete = london_complete.rename(columns=str.lower)

In [ ]:
# Rename some columns to more logical names without changing underlying data dictionary.
london_complete = london_complete.rename(columns={'location_x': 'location', 
                                                  'survey_wave_(year)': 'survey_year'})

In [ ]:
# Drop all rows with duplicate values
london_complete.drop_duplicates()

# All duplicate rows dropped
# Returns the number of rows expected
# Data now is cleaned
# Data will be saved
# No outlier detection carried out as data sourced from Tfl and was presented semi wrangled
# If further analysis doesnt obviously make sense will return for outlier detection

In [ ]:
# Count unique values in each column
for col in london_complete:
  print(col,": ", london_complete[col].nunique())

# Survey data over 8 years
# Across 1258 counters
# Across 1188 streets
# Across 33 boroughs in London

In [ ]:
# Store the combined dataframe as a new CSV for backup
london_complete.to_csv('london_count_and_site_201022.csv')

# Introducing Data for Biking Infrastructure

Beyond the count data supplied for this project, it will be useful to investigate the cycling infrastructure in London. To this end, the [TfL](https://cycling.data.tfl.gov.uk/ "Cycling Infrastructure") produces data on a range of cycling assets especially for the purpose of analysis.

Data on the following types of infrastructure are published by the TfL and will be analysed here:

- Cycle parking, including the type and capacity of parking
- Restricted Points – points where people cycling will have to dismount Paths through parks and other green spaces that can, and cannot, be cycled on
- Signage - Signed cycle routes and other wayfinding
- Signals - early-release signals at junctions
- Traffic calming, including the location of all speed humps in Greater London
- Advanced stop lines – boxes at junctions for people cycling
- Restricted route - Modal filters and traffic gates which allow cycles to pass but restrict car traffic
- Signalised crossings for cycles
- Cycle lanes and tracks – including whether they are segregated or painted lanes


In [ ]:
# Reading datapoint CSVs.
cycle_park = pd.read_csv('cycle_parking.csv')
rct_point = pd.read_csv('restricted_point.csv')
signage = pd.read_csv('signage.csv')
signal = pd.read_csv('signal.csv')
traf_calm = pd.read_csv('traffic_calming.csv')

# Reading datalines CSVs.
asl = pd.read_csv('advanced_stop_line.csv')
rct_route = pd.read_csv('restricted_route.csv')
crossing = pd.read_csv('crossing.csv')
cyc_lane = pd.read_csv('cycle_lane_track.csv')

In [ ]:
# Removing 'properties/' from the column names.
cycle_park.columns = cycle_park.columns.str.replace('properties/','')
rct_point.columns = rct_point.columns.str.replace('properties/','')
signage.columns = signage.columns.str.replace('properties/','')
signal.columns = signal.columns.str.replace('properties/','')
traf_calm.columns = traf_calm.columns.str.replace('properties/','')

asl.columns = asl.columns.str.replace('properties/','')
rct_route.columns = rct_route.columns.str.replace('properties/','')
crossing.columns = crossing.columns.str.replace('properties/','')
cyc_lane.columns = cyc_lane.columns.str.replace('properties/','')

In [ ]:
# Removing 'properties.' from the column names.
cycle_park.columns = cycle_park.columns.str.replace('properties.','')
rct_point.columns = rct_point.columns.str.replace('properties.','')
signage.columns = signage.columns.str.replace('properties.','')
signal.columns = signal.columns.str.replace('properties.','')
traf_calm.columns = traf_calm.columns.str.replace('properties.','')

asl.columns = asl.columns.str.replace('properties.','')
rct_route.columns = rct_route.columns.str.replace('properties.','')
crossing.columns = crossing.columns.str.replace('properties.','')
cyc_lane.columns = cyc_lane.columns.str.replace('properties.','')

In [ ]:
# Removing 'geometry/' from the column names.
cycle_park.columns = cycle_park.columns.str.replace('geometry/','')
rct_point.columns = rct_point.columns.str.replace('geometry/','')
signage.columns = signage.columns.str.replace('geometry/','')
signal.columns = signal.columns.str.replace('geometry/','')
traf_calm.columns = traf_calm.columns.str.replace('geometry/','')

asl.columns = asl.columns.str.replace('geometry/','')
rct_route.columns = rct_route.columns.str.replace('geometry/','')
crossing.columns = crossing.columns.str.replace('geometry/','')
cyc_lane.columns = cyc_lane.columns.str.replace('geometry/','')

In [ ]:
# Removing 'geometry.' from the column names.
cycle_park.columns = cycle_park.columns.str.replace('geometry.','')
rct_point.columns = rct_point.columns.str.replace('geometry.','')
signage.columns = signage.columns.str.replace('geometry.','')
signal.columns = signal.columns.str.replace('geometry.','')
traf_calm.columns = traf_calm.columns.str.replace('geometry.','')

asl.columns = asl.columns.str.replace('geometry.','')
rct_route.columns = rct_route.columns.str.replace('geometry.','')
crossing.columns = crossing.columns.str.replace('geometry.','')
cyc_lane.columns = cyc_lane.columns.str.replace('geometry.','')

In [ ]:
# Changing column names to all lower.
cycle_park = cycle_park.rename(columns=str.lower)
rct_point = rct_point.rename(columns=str.lower)
signage = signage.rename(columns=str.lower)
signal = signal.rename(columns=str.lower)
traf_calm = traf_calm.rename(columns=str.lower)

asl = asl.rename(columns=str.lower)
rct_route = rct_route.rename(columns=str.lower)
crossing = crossing.rename(columns=str.lower)
cyc_lane = cyc_lane.rename(columns=str.lower)

In [ ]:
# Dropping unneccesary columns and renaming columns where necessary.
# Extracting latitude and Longitude from a combined column.

splits = signage["coordinates"].str.split(",", n = 1, expand = True)
signage["longitude"] = splits[0]
signage["latitude"] = splits[1]
signage = signage.drop(columns=["coordinates"])
signage["longitude"] = signage["longitude"].str.strip('[').astype(float)
signage["latitude"] = signage["latitude"].str.strip(']').astype(float)

signage = signage.drop(columns=['unnamed: 0','type','type','ss_routen','ss_access', 'photo1_url', 'photo2_url'])
signage = signage.rename(columns={'ss_road':'Road Marking',
                                  'ss_patch':'Coloured Patch on Surface',
                                  'ss_colour':'Colour of Patch',
                                  'ss_facing':'Facing Off-side',
                                  'ss_nocyc':'No Cycling',
                                  'ss_noveh':'No Vehicles',
                                  'ss_circ':'Circular Sign',
                                  'ss_exempt':'Exemption',
                                  'ss_noleft':'No Left Turn Exception',
                                  'ss_norigh':'No Right Turn Exception',
                                  'ss_left':'Compulsory Turn Left Exception',
                                  'ss_right':'Compulsory Turn Right exception',
                                  'ss_noexce':'No Straight Ahead Exception',
                                  'ss_dismou':'Cyclists Dismount',
                                  'ss_end':'End of Route',
                                  'ss_cycsmb':'Cycle Symbol',
                                 })


# Restricted point dataframe.
rct_point = rct_point.drop(columns=['type','type','photo1_url', 'photo2_url'])
rct_point = rct_point.rename(columns={'coordinates/0': 'longitude', 
                                      'coordinates/1': 'latitude',
                                     'rst_steps':'Steps',
                                      'rst_lift':'Lift',
                                      'res_pedest':'Pedestrian-Only Route',
                                      'res_bridge':'Pedestrian Bridge',
                                      'res_tunnel':'Pedestrian Tunnel',
                                      'res_steps':'Steps','res_lift':'Lift'
                                     })

# Cycle Park dataframe.
cycle_park = cycle_park.drop(columns=['type','type','photo1_url', 'photo2_url'])
cycle_park = cycle_park.rename(columns={'coordinates/0': 'longitude', 
                                        'coordinates/1': 'latitude', 
                                        'prk_carr':'Carriageway',
                                        'prk_cover':'Covered',
                                        'prk_secure':'Secure',
                                        'prk_locker':'Locker',
                                        'prk_sheff':'Sheffield Stand',
                                        'prk_mstand':'"M" stand',
                                        'prk_pstand':'"P" stand',
                                        'prk_hoop':'Cyclehoop',
                                        'prk_post':'Post',
                                        'prk_buterf':'Butterfly',
                                        'prk_wheel':'Wheel Rack',
                                        'prk_hangar':'Bike Hangar',
                                        'prk_tier':'Two Tier',
                                        'prk_other':'Other',
                                        'prk_provis':'Provision',
                                        'prk_cpt':'Capacity','':''
                                        })


# Signal dataframe.
signal = signal.drop(columns=['type','type','photo1_url', 'photo2_url'])
signal = signal.rename(columns={'coordinates/0': 'longitude', 
                                'coordinates/1': 'latitude',
                               'sig_head':'Cycle Signal Head',
                                'sig_separa':'Separate Stage for Cyclists',
                                'sig_early':'Early Release',
                                'sig_twostg':'Two Stage Turn',
                                'sig_gate':'Cycle/Bus Gate'
                               })



# Traffic calming dataframe.

splits2 = traf_calm["coordinates"].str.split(",", n = 1, expand = True)
traf_calm["longitude"] = splits2[0]
traf_calm["latitude"] = splits2[1]
traf_calm = traf_calm.drop(columns=["coordinates"])
traf_calm["longitude"] = traf_calm["longitude"].str.strip('[').astype(float)
traf_calm["latitude"] = traf_calm["latitude"].str.strip(']').astype(float)


traf_calm = traf_calm.drop(columns=['unnamed: 0','type','type','photo1_url', 'photo2_url'])
signal = signal.rename(columns={'trf_raised':'Raised Table',
                                'trf_entry':'Side Road Entry Treatment',
                                'trf_cushi':'Speed Cushions',
                                'trf_hump':'Speed Hump','trf_sinuso':'Sinusoidal',
                                'trf_barier':'Barrier',
                                'trf_narow':'Carriageway Narrowing',
                                'trf_calm':'Other'
                                })


# Advanced stop lanes dataframe.
asl = asl.drop(asl.iloc[:, 14:31], axis = 1)
asl = asl.drop(asl.iloc[:, 0:1], axis = 1)
asl = asl.rename(columns={'coordinates/0/0': 'longitude_start', 
                          'coordinates/0/1': 'latitude_start', 
                          'coordinates/1/0': 'longitude_end',
                          'coordinates/1/1': 'latitude_end',
                          'asl_fdr':'Feeder Lane',
                          'asl_fdrlft':'Feeder Lane on Left',
                          'asl_fdcent':'Feeder Lane in Centre',
                          'asl_fdrigh':'Feeder Lane on Right',
                          'asl_shared':'Shared Nearside Lane',
                          'asl_colour':'Colour'})


# Restricted Route dataframe.

splits = rct_route["coordinates"].str.split(",", n = 1, expand = True)
rct_route["longitude"] = splits[0]
rct_route["latitude"] = splits[1]
rct_route = rct_route.drop(columns=["coordinates"])
rct_route["longitude"] = rct_route["longitude"].str.strip('[').astype(float)
rct_route["latitude"] = rct_route["latitude"].str.strip(']').astype(float)
rct_route = rct_route.drop(['unnamed: 0','type','type','photo1_url','photo2_url'], axis = 1)

# Crossing dataframe.
crossing = crossing.drop(crossing.iloc[:, 14:56], axis = 1)
crossing = crossing.drop(crossing.iloc[:, 0:1], axis = 1)
crossing = crossing.rename(columns={'coordinates/0/0': 'longitude_start', 
                          'coordinates/0/1': 'latitude_start', 
                          'coordinates/1/0': 'longitude_end',
                          'coordinates/1/1': 'latitude_end',
                          'crs_signal':'Signal-Controlled Crossing',
                          'crs_segreg':'Segregated Cycles and Pedestrians',
                          'crs_cygap':'Cycle Gap',
                          'crs_pedest':'Pedestrian-Only Crossing',
                          'crs_level':'Level Crossing'
                                   })


# Cycle Lane dataframe.

splits = cyc_lane["coordinates"].str.split(",", n = 2, expand = True)
cyc_lane["longitude"] = splits[0]
cyc_lane["latitude"] = splits[1]
cyc_lane = cyc_lane.drop(columns=["coordinates"])
cyc_lane["longitude"] = cyc_lane["longitude"].str.strip('[').astype(float)
cyc_lane["latitude"] = cyc_lane["latitude"].str.strip(']').astype(float)


cyc_lane = cyc_lane.drop(['unnamed: 0','type','type','photo1_url','photo2_url'], axis = 1)
cyc_lane = cyc_lane.rename(columns={'clt_carr':'On-off Carriageway',
                                    'clt_segreg':'Segregated Lane/Track',
                                    'clt_stepp':'Stepped Lane/Track',
                                    'clt_parseg':'Partially Segregated Lane/Track',
                                    'clt_shared':'Shared Lane or Footway',
                                    'clt_mandat':'Mandatory Cycle Lane',
                                    'clt_advis':'Advisory Cycle Lane',
                                    'clt_priori':'Cycle Lane/Track Priority',
                                    'clt_contra':'Contraflow Lane/Track',
                                    'clt_bidire':'Bi-directional',
                                    'clt_cbypas':'Cycle Bypass',
                                    'clt_bbypas':'Continuous Cycle Facilities at Bus Stop',
                                    'clt_parkr':'Park Route','clt_waterr':'Waterside Route',
                                    'clt_ptime':'Part-time (if true) or Full-time (if false)',
                                    'clt_access':'Access Times','clt_colour':'Colour'
                                    })

In [ ]:
# Replacing null values.
cyc_lane = cyc_lane.fillna(0)
rct_route = rct_route.fillna(0)
signage = signage.fillna(0)
traf_calm = traf_calm.fillna(0)
crossing = crossing.fillna(0)
asl = asl.fillna(0)
signal = signal.fillna(0)
cycle_park = cycle_park.fillna(0)
rct_point = rct_point.fillna(0)

In [ ]:
# Parse London Df Dates into appropriate format.

# Convert to datetime format.
cyc_lane['svdate'] = pd.to_datetime(cyc_lane['svdate'], dayfirst=True, yearfirst=False, errors = 'coerce')
rct_route['svdate'] = pd.to_datetime(rct_route['svdate'], dayfirst=True, yearfirst=False , errors = 'coerce')
signage['svdate'] = pd.to_datetime(signage['svdate'], dayfirst=True, yearfirst=False, errors = 'coerce')
traf_calm['svdate'] = pd.to_datetime(traf_calm['svdate'], dayfirst=True, yearfirst=False , errors = 'coerce')
crossing['svdate'] = pd.to_datetime(crossing['svdate'], dayfirst=True, yearfirst=False, errors = 'coerce')
asl['svdate'] = pd.to_datetime(asl['svdate'], dayfirst=True, yearfirst=False, errors = 'coerce')
signal['svdate'] = pd.to_datetime(signal['svdate'], dayfirst=True, yearfirst=False, errors = 'coerce')
cycle_park['svdate'] = pd.to_datetime(cycle_park['svdate'], dayfirst=True, yearfirst=False, errors = 'coerce')
rct_point['svdate'] = pd.to_datetime(rct_point['svdate'], dayfirst=True, yearfirst=False, errors = 'coerce')

# Sort DataFrame by chronological order.
cyc_lane = cyc_lane.sort_values(['svdate'])
rct_route = rct_route.sort_values(['svdate'])
signage = signage.sort_values(['svdate'])
traf_calm = traf_calm.sort_values(['svdate'])
crossing = crossing.sort_values(['svdate'])
asl = asl.sort_values(['svdate'])
signal = signal.sort_values(['svdate'])
cycle_park = cycle_park.sort_values(['svdate'])
rct_point = rct_point.sort_values(['svdate'])


# Define day of the week in English and add back as a column.
cyc_lane['Day_of_week'] = cyc_lane['svdate'].dt.day_name()
rct_route['Day_of_week'] = rct_route['svdate'].dt.day_name()
signage['Day_of_week'] = signage['svdate'].dt.day_name()
traf_calm['Day_of_week'] = traf_calm['svdate'].dt.day_name()
crossing['Day_of_week'] = crossing['svdate'].dt.day_name()
asl['Day_of_week'] = asl['svdate'].dt.day_name()
signal['Day_of_week'] = signal['svdate'].dt.day_name()
cycle_park['Day_of_week'] = cycle_park['svdate'].dt.day_name()
rct_point['Day_of_week'] = rct_point['svdate'].dt.day_name()

# Pass Month into a new column.
cyc_lane['month'] = cyc_lane['svdate'].dt.month
rct_route['month'] = rct_route['svdate'].dt.month
signage['month'] = signage['svdate'].dt.month
traf_calm['month'] = traf_calm['svdate'].dt.month
crossing['month'] = crossing['svdate'].dt.month
asl['month'] = asl['svdate'].dt.month
signal['month'] = signal['svdate'].dt.month
cycle_park['month'] = cycle_park['svdate'].dt.month
rct_point['month'] = rct_point['svdate'].dt.month

# Pass Year into a new column.
cyc_lane['year'] = cyc_lane['svdate'].dt.year
rct_route['year'] = rct_route['svdate'].dt.year
signage['year'] = signage['svdate'].dt.year
traf_calm['year'] = traf_calm['svdate'].dt.year
crossing['year'] = crossing['svdate'].dt.year
asl['year'] = asl['svdate'].dt.year
signal['year'] = signal['svdate'].dt.year
cycle_park['year'] = cycle_park['svdate'].dt.year
rct_point['year'] = rct_point['svdate'].dt.year

In [ ]:
# Converting boolean values to integers to sum the infrastructure.
cyc_lane.replace({False: 0, True: 1}, inplace=True)
rct_route.replace({False: 0, True: 1}, inplace=True)
signage.replace({False: 0, True: 1}, inplace=True)
traf_calm.replace({False: 0, True: 1}, inplace=True)
crossing.replace({False: 0, True: 1}, inplace=True)
asl.replace({False: 0, True: 1}, inplace=True)
signal.replace({False: 0, True: 1}, inplace=True)
cycle_park.replace({False: 0, True: 1}, inplace=True)
rct_point.replace({False: 0, True: 1}, inplace=True)

In [ ]:
rct_point.info()

In [ ]:
# Adding calculated column to sum up all infrastructure.
cyc_lane['infra_sum'] = cyc_lane.iloc[:, 2:17].sum(axis=1)
rct_route['infra_sum'] = rct_route.iloc[:, 2:32].sum(axis=1)
signage['infra_sum'] = signage.iloc[:, 2:29].sum(axis=1)
traf_calm['infra_sum'] = traf_calm.iloc[:, 2:9].sum(axis=1)
crossing['infra_sum']= crossing.iloc[:, 6:10].sum(axis=1)
asl['infra_sum']= asl.iloc[:, 6:10].sum(axis=1)
signal['infra_sum']= signal.iloc[:, 4:8].sum(axis=1)
cycle_park['infra_sum'] = cycle_park.iloc[:, 4:19].sum(axis=1)
rct_point['infra_sum'] = rct_point.iloc[:, 4:5].sum(axis=1)

In [ ]:
# Creating a time series grouping by year.
# Time series is sum of all infrastructure per type of infrastructure.
cyc_lane_grpby = cyc_lane.groupby(['year'], as_index=False)['infra_sum'].sum()
rct_route_grpby = rct_route.groupby(['year'], as_index=False)['infra_sum'].sum()
signage_grpby = signage.groupby(['year'], as_index=False)['infra_sum'].sum()
crossing_grpby = crossing.groupby(['year'], as_index=False)['infra_sum'].sum()
asl_grpby = asl.groupby(['year'], as_index=False)['infra_sum'].sum()
signal_grpby = signal.groupby(['year'], as_index=False)['infra_sum'].sum()
cycle_park_grpby = cycle_park.groupby(['year'], as_index=False)['infra_sum'].sum()
rct_point_grpby = rct_point.groupby(['year'], as_index=False)['infra_sum'].sum()
traf_calm_grpby = traf_calm.groupby(['year'], as_index=False)['infra_sum'].sum()

In [ ]:
# Grouping the infrastructure data by borough only.
cyc_lane_bor_grpby = cyc_lane.groupby(['borough'], as_index=False)['infra_sum'].sum()
rct_route_bor_grpby = rct_route.groupby(['borough'], as_index=False)['infra_sum'].sum()
signage_bor_grpby = signage.groupby(['borough'], as_index=False)['infra_sum'].sum()
crossing_bor_grpby = crossing.groupby(['borough'], as_index=False)['infra_sum'].sum()
signal_bor_grpby = signal.groupby(['borough'], as_index=False)['infra_sum'].sum()
cycle_park_bor_grpby = cycle_park.groupby(['borough'], as_index=False)['infra_sum'].sum()
rct_point_bor_grpby = rct_point.groupby(['borough'], as_index=False)['infra_sum'].sum()
traf_calm_bor_grpby = traf_calm.groupby(['borough'], as_index=False)['infra_sum'].sum()

In [ ]:
# Breaking down the infrastructure by borough & year.
cyc_lane_yr_bor_grpby = cyc_lane.groupby(['year','borough'], as_index=False)['infra_sum'].sum()
rct_route_yr_bor_grpby = rct_route.groupby(['year','borough'], as_index=False)['infra_sum'].sum()
signage_yr_bor_grpby = signage.groupby(['year','borough'], as_index=False)['infra_sum'].sum()
crossing_yr_bor_grpby = crossing.groupby(['year','borough'], as_index=False)['infra_sum'].sum()

signal_yr_bor_grpby = signal.groupby(['year','borough'], as_index=False)['infra_sum'].sum()
cycle_park_yr_bor_grpby = cycle_park.groupby(['year','borough'], as_index=False)['infra_sum'].sum()
rct_point_yr_bor_grpby = rct_point.groupby(['year','borough'], as_index=False)['infra_sum'].sum()
traf_calm_yr_bor_grpby = traf_calm.groupby(['year','borough'], as_index=False)['infra_sum'].sum()

In [ ]:
# Visualising the top 10 boroughs with the most cycle lanes.
cyc_lane_largest_ten = cyc_lane_bor_grpby.nlargest(10,'infra_sum')

# Plotting.
cyc_lane_largest_ten.plot.barh(x = "borough", 
                               y = "infra_sum", 
                               title = "Top 10 boroughs for highest number of Cycle Lanes", 
                               legend = False, 
                               xlabel = 'Borough', 
                               ylabel = 'No. of Cycle Lanes')
plt.show()

In [ ]:
# Visualising the top 10 boroughs with the most Restricted Routes.
rct_route_largest_ten = rct_route_bor_grpby.nlargest(10, 'infra_sum')

# Plotting.

rct_route_largest_ten.plot.barh(x="borough", 
                                y="infra_sum", 
                                title="Top 10 boroughs for highest number of  Restricted Routes", 
                                legend=False, 
                                xlabel='Borough', 
                                ylabel= 'No. of Restricted Routes' )

plt.show()

In [ ]:
# Visualising the top 10 boroughs with the most signage.
signage_bor_largest_ten = signage_bor_grpby.nlargest(10,'infra_sum')

# Plotting.
signage_bor_largest_ten.plot.barh(x="borough", 
                                  y="infra_sum", 
                                  title="Top 10 boroughs for highest number of Signages", 
                                  legend=False, 
                                  xlabel='Borough', 
                                  ylabel= 'No. of Signages')
plt.show()

In [ ]:
# Visualising the top 5 boroughs with the most signalised crossings for cycles.
crossing_bor_largest_ten = crossing_bor_grpby.nlargest(10,'infra_sum')

# Plotting.
crossing_bor_largest_ten.plot.barh(x="borough", 
                                   y="infra_sum", 
                                   title="Top 10 boroughs for highest number of Signalised Crossings", 
                                   legend=False, 
                                   xlabel='Borough', 
                                   ylabel='No. of Signals' )
plt.show()

In [ ]:
# Visualising the top 10 boroughs with the most Early release signals at junctions for cyclists.
signal_bor_largest_ten = signal_bor_grpby.nlargest(10,'infra_sum')

# Plotting.
signal_bor_largest_ten.plot.barh(x="borough", 
                                 y="infra_sum", 
                                 title="Top 10 boroughs for highest number of Early Release Signals", 
                                 legend=False, 
                                 xlabel='Borough', 
                                 ylabel='No. of Signals')
plt.show()

In [ ]:
# Visualising the top 10 boroughs with the most cycle parking.
cycle_park_bor_largest_ten = cycle_park_bor_grpby.nlargest(10,'infra_sum')

# Plotting.
cycle_park_bor_largest_ten.plot.barh(x="borough", 
                                     y="infra_sum", 
                                     title="Top 10 boroughs for highest number of Cycle Parkings", 
                                     legend=False, 
                                     xlabel='borough', 
                                     ylabel='No. of Cycle Parkings' )

plt.show()

In [ ]:
# Visualising the top 10 boroughs with the most restricted points.
rct_point_bor_largest_ten = rct_point_bor_grpby.nlargest(10,'infra_sum')

# Plotting.
rct_point_bor_largest_ten.plot.barh(x="borough", 
                                    y="infra_sum", 
                                    title="Top 10 boroughs with highest number of Restricted Points", 
                                    legend=False, 
                                    xlabel='borough',
                                    ylabel='No. of Restricted points' )
plt.show()

In [ ]:
# Visualising the top 10 boroughs with the most Traffic Calming.
traf_calm_bor_largest_ten = traf_calm_bor_grpby.nlargest(10, 'infra_sum')

# Plotting.
traf_calm_bor_largest_ten.plot.barh(x="borough", 
                                    y="infra_sum", 
                                    title="Top 10 for Traffic Calming", 
                                    legend=False, 
                                    xlabel='borough', 
                                    ylabel='No. of Traffic Calming Measures' )

plt.show()

# Introducing additional Data from independent research

Absent timeseries data covering the full period of this analysis on the development of cycling infrastructure in London, some other variables will be analysed which could have a meaningfully quantifiable relationship with cycling uptake. 

- [Private Car Ownership](https://data.london.gov.uk/dataset/licensed-vehicles-numbers-borough "Private Car Ownership") 
- [Traffic Flow](https://data.london.gov.uk/dataset/traffic-flows-borough "Traffic Flow")

Furthermore, to baseline and contextualise the cycling count data, population time series for [London](https://www.macrotrends.net/cities/22860/london/population "London"), [New York](https://www.macrotrends.net/states/new-york/population "New York") and [Sydney](https://www.macrotrends.net/cities/206167/sydney/population "Sydney") are also being introduced.

In [ ]:
# Introduce new data.
# Traffic and Car Ownership data.
car_own = pd.read_csv('private_cars_london.csv')
traffic_flow = pd.read_csv('traffic_flow_borough.csv')

In [ ]:
# Population & cycling data.
pop_london = pd.read_csv('London_population.csv')
pop_ny = pd.read_csv('New_York_population.csv')
pop_sydney = pd.read_csv('Sydney_population.csv')
london_complete = pd.read_csv('london_count_and_site_201022.csv')

In [ ]:
# Change all column names to lower case to ease recalling them for analysis.
car_own = car_own.rename(columns=str.lower)
traffic_flow = traffic_flow.rename(columns=str.lower)
pop_london = pop_london.rename(columns=str.lower)
pop_ny = pop_ny.rename(columns=str.lower)
pop_sydney = pop_sydney.rename(columns=str.lower)

In [ ]:
# Rename columns to remove space in column names.
traffic_flow.columns = traffic_flow.columns.str.replace(' ','_')
car_own.columns = car_own.columns.str.replace(' ','_')
pop_london.columns = pop_london.columns.str.replace(' ','_')
pop_ny.columns = pop_ny.columns.str.replace(' ','_')
pop_sydney.columns = pop_sydney.columns.str.replace(' ','_')

In [ ]:
# View Sample.
pop_london.head()

In [ ]:
# View Sample.
pop_ny.head()

In [ ]:
# View Sample.
pop_sydney.head()

## Shaping and cleaning the data for analysis

In [ ]:
# Change to datatime format.
pop_london['date'] = pd.to_datetime(pop_london['date'], format='%d/%m/%Y')
pop_ny['date'] = pd.to_datetime(pop_ny['date'], format='%d/%m/%Y')
pop_sydney['date'] = pd.to_datetime(pop_sydney['date'], format='%d/%m/%Y')

In [ ]:
# Need to create a Year Column in each DF.
# Will use this to merge the population data.
pop_london['year'] = pop_london['date'].dt.year
pop_ny['year'] = pop_ny['date'].dt.year
pop_sydney['year'] = pop_sydney['date'].dt.year

In [ ]:
# Drop date column from each population DF.
pop_london.drop('date', axis=1, inplace=True)
pop_ny.drop('date', axis=1, inplace=True)
pop_sydney.drop('date', axis=1, inplace=True)

In [ ]:
# View Sample.
car_own.head(5)

# Shows absolute numbers of cars registered in private ownership over time in each borough.

In [ ]:
# View Sample.
traffic_flow.head(5)

# Unit of measure is in millions & kilometers.
# Shows million vehicle kilometers travelled by all cars through each borough over time. 

In [ ]:
# Merge the population dfs.
# Merge the traffic flow and car ownership details.
dfs = [pop_london, pop_ny, pop_sydney]
all_pop = ft.reduce(lambda left, right: pd.merge(left, right, on='year'), dfs)

In [ ]:
# View output.
all_pop

In [ ]:
# Rename Columns. 
all_pop.rename(columns={'_population_x': 'london_pop', '_population_y': 'ny_pop', 
                       '_population': 'sydney_pop'}, inplace=True)

In [ ]:
# Create a column showing Y-o-Y % change in population for each city.
all_pop['ldn_pop_change'] = pd.DataFrame.pct_change(all_pop['london_pop'])*100
all_pop['ny_pop_change'] = pd.DataFrame.pct_change(all_pop['ny_pop'])*100
all_pop['sydney_pop_change'] = pd.DataFrame.pct_change(all_pop['sydney_pop'])*100

In [ ]:
# Create log values for population to reduce the scalar impact on visualisations.
all_pop['log_london_pop'] = np.log(all_pop['london_pop'])
all_pop['log_ny_pop'] = np.log(all_pop['ny_pop'])
all_pop['log_sydney_pop'] = np.log(all_pop['sydney_pop'])

In [ ]:
# Shape the df for further analysis.
all_pop1 = pd.melt(all_pop, id_vars =['year'], value_vars =['log_london_pop', 'log_ny_pop', 'log_sydney_pop'])

In [ ]:
# Rename some columns to align across analysis.
all_pop1 = all_pop1.rename(columns={'variable': 'location', 'value': 'population(log)', 
                                   'population': 'location'})

In [ ]:
# Replace column values to align analysis.
all_pop1 = all_pop1.replace(['log_london_pop','log_ny_pop',
                             'log_sydney_pop'],['london', 'new_york', 'sydney'])

In [ ]:
# Look now at how cycling uptake has evolved in London over the same period.
# Introduce the merged london data set and recall the metadata.
london_complete.info()

In [ ]:
# Look now at how cycling uptake has evolved in London over the same period.
# Ensure date is in appropriate format.
london_complete['survey_date'] = pd.to_datetime(london_complete.survey_date)

# Aggregate by year.
# Insert column year to run aggregation function.
london_complete['year'] = london_complete['survey_date'].dt.year

In [ ]:
# Fill all NaN values with 0.
london_complete.fillna(0)

In [ ]:
# Simplify df columns.
london_complete=london_complete.drop(['Unnamed: 0', 'survey_year', 'direction', 
                                      'surveydescription', 'total_cycles'], axis=1)

In [ ]:
# Insert a total count in outer london.
london_complete['total_outer'] = london_complete['number_of_male_cycles']+london_complete['number_of_female_cycles']+london_complete['number_of_unknown_cycles']

In [ ]:
# Insert a total count for inner and central london.
london_complete['total_inn_cen'] = london_complete['number_of_private_cycles']+london_complete['number_of_cycle_hire_bikes']

In [ ]:
# Fill all NaN values with 0.
london_complete['total_outer'] = london_complete['total_outer'].fillna(0)
london_complete['total_inn_cen'] = london_complete['total_inn_cen'].fillna(0)

In [ ]:
# Insert a global total uptake column for cycling counts in all areas of london.
london_complete['total_uptake'] = london_complete['total_outer']+london_complete['total_inn_cen'] 

In [ ]:
# Group by year to see change of total cycling uptake YoY.
df= london_complete.groupby(['year'], as_index=False)['total_uptake'].sum()

In [ ]:
# Put all numbers on log scale to supress scalar affect.
df['log_total_uptake_ldn'] = np.log(df['total_uptake']) 

In [ ]:
# Clean the df.
df=df.drop(['total_uptake'], axis=1)

In [ ]:
# Granularity on how total_uptake evolved in the 3 parts of London across common variables.
df1= london_complete.groupby(['year', 'functional_cycling_area', 'weather', 'period', 
                             'day_of_week', 'month_name', 'month', 'season'], as_index=False)['total_uptake'].sum()

In [ ]:
# Visualise the changes.
# Put all numbers on log scale to supress scalar affect.
df1['log_total_uptake_ldn'] = np.log(df1['total_uptake']) 

In [ ]:
# Get overview of how cycling uptake has progressed over the years in New York.
df3 = ny_counts.groupby(['year'], as_index=False)['counts'].sum()

In [ ]:
# Put all numbers on log scale to supress scalar affects.
df3['log_total_uptake_ny'] = np.log(df3['counts']) 

In [ ]:
# Clean the df.
df3=df3.drop(['counts'], axis=1)

In [ ]:
# Change all column names to lower case to ease recalling them for analysis.
sydney_counts = sydney_counts.rename(columns=str.lower)

In [ ]:
# Get overview of how cycling uptake has progressed over the years in Sydney.
df4 = sydney_counts.groupby(['year'], as_index=False)['totalcount'].sum()

In [ ]:
# Put all numbers on log scale to supress scalar affects.
df4['log_total_uptake_sydney'] = np.log(df4['totalcount']) 

In [ ]:
# Df4 shows strange dip in 2018 numbers.
# Investigate further.
df4a = sydney_counts.groupby(["year","month"], as_index=False)['totalcount'].sum()
df4a

# March 2018 data missing
# Oct 2018 has duplicate

In [ ]:
# Drop duplicate.
df4a = df4a.drop([17])

In [ ]:
# Reshape for correction of missing values.
df4a = df4a.pivot(index='year', columns='month', values='totalcount')
df4a.as_index=False
# This inserts March.

In [ ]:
# Replace missing value with interpolated value. 
df4a = df4a.interpolate()

In [ ]:
# Reshape the dataframe for further analysis and alignment with other cities.
df4a['totalcount'] = df4a['March'] + df4a["October"]
df4a['log_total_uptake_sydney'] = np.log(df4a['totalcount'])
df4a=df4a.drop(['March', 'October', 'totalcount'], axis=1)

# Reset the index.
df4 = df4a.reset_index()

In [ ]:
# Plot all regions/cities of interest in one chart.
# Merge the dataframes.
dfs = [df, df3, df4]
on = ['year']
all_counts = ft.reduce(lambda left, right: pd.merge(left, right, on= on), dfs)

In [ ]:
# Melt the dataframe to plot all three cities into one chart.
df5 = pd.melt(all_counts, id_vars =['year'], value_vars =['log_total_uptake_ldn', 
                                                          'log_total_uptake_ny', 
                                                          'log_total_uptake_sydney'])

In [ ]:
# Rename some columns to align across analysis
df5 = df5.rename(columns={'variable': 'location', 'value': 'cycling_uptake_log'})

In [ ]:
# Replace location values to align across analysis
df5 = df5.replace(['log_total_uptake_ldn','log_total_uptake_ny',
                   'log_total_uptake_sydney'],['london', 'new_york', 'sydney'])

In [ ]:
# Merge Population & Cycling Uptake Data across all 3 cities over the period being analysed
dfs = [df5, all_pop1]
on = ['year', 'location']
city_counts = ft.reduce(lambda left, right: pd.merge(left, right, on=on), dfs)

# View output
city_counts

In [ ]:
# Rename columns to remove brackets
city_counts = city_counts.rename(columns={'population(log)': 'population_log'})

In [ ]:
# Create a new pivot for cycling uptake
# This will help visualise the cycling and population values in one chart
city_counts_piv_CU = city_counts.pivot(index='year', columns='location', values='cycling_uptake_log')
city_counts_piv_CU = city_counts_piv_CU.reset_index()

#View output
city_counts_piv_CU

In [ ]:
# Do the same for population
city_counts_piv_pop = city_counts.pivot(index= 'year', columns= 'location', values='population_log')
city_counts_piv_pop = city_counts_piv_pop.reset_index()

#View output
city_counts_piv_pop

# Visual Insights from the quantitative data

## Comparative Analysis between Cities

### How have population & cycling uptake changed?

In [ ]:
# Visualise if cycling uptake in each city is proportionate to its population.
fig, ax1 = plt.subplots()
fig.set_size_inches(10, 5)

# twinx() function is required to have two 'y-axis', for two 'x-axis' we will use twiny() function.
ax2 = ax1.twinx()

# Plotting both DFs.
city_counts_piv_CU.plot(x='year',y=['london','new_york','sydney'] ,marker='o', ax=ax1)
city_counts_piv_pop.plot(x='year',y=['london','new_york','sydney'], linestyle='dashed', ax=ax2)

# Plotting attributes.
ax1.set_title('Comparison of Population vs Cycling Uptake')
ax1.set_xlabel('Year')
ax1.set_ylabel('Population')
ax2.set_ylabel('Cycling Uptake')
ax1.legend(title='Cycling' ,
           loc='center', 
           bbox_to_anchor=(-0.15,0.4), 
           labelspacing = 0.75, 
           frameon=False)
ax2.legend(title='Population', 
           loc='center', 
           bbox_to_anchor=(-0.15,0.1), 
           labelspacing = 0.75, 
           frameon=False)

ax1.grid(False)
ax2.grid(False)

plt.tight_layout()
plt.show()

# London shows strong decline which is due to the fact counting was stopped in 2020 for long periods.
# This was seen is the calendar heatmap for missing data.
# New York Cycling uptake peaked in 2016 and has been declining very slowly since.
# London uptake (bar 2020) also very stable except good growth in the first few years.
# Sydney uptake has been on a secular decline.
# 2020/21 decline in Sydney possibly related to a change in count methodology.
# There was a sharp decline in frequency of count data in 2020/21 in Sydney.
# This was seen in the missing data distribution analysis.

## Comparative Analysis between London's regions

In [ ]:
# Visualise change in uptake in the 3 parts of London over time.
g=sns.lineplot(x="year", 
               y="log_total_uptake_ldn", 
               data=df1, 
               hue='functional_cycling_area', ci=None)
g.set_title('Cycling uptake in London & its regions')
g.set_ylabel('Cycling Uptake (log scale)')
g.set_xlabel('Year')

plt.legend(bbox_to_anchor=(-0.15, 0.1), 
           borderaxespad=0, 
           frameon=False)

plt.grid(b=None)

# Central London uptake stable although slowly rising beyond pre-pandemic peak.
# Strong growth in uptake between 2019 and 2020 in the regions.
# Similar but less muted trend in Central London.
# Sharp decline in regional uptake in the post pandemic period.
# Could it mean leisure cyclists returning back to work and not cycling any longer?

## Macro view of London's total cycling Uptake

In [ ]:
# Visualise the change in all areas of London combined.
# Visualising on its own to manage scalar effects more tightly. 
# London is city of most interest.
# Worth look at it on its own and also versus other cities.
# Create the line chart to visualise the change.
plt.plot(df['year'], df['log_total_uptake_ldn'])

# Adding the aesthetics.
plt.title("YoY cycling uptake in London across all regions, 2014-2021")
plt.xlabel('Year')
plt.ylabel('Cycling uptake (log scale)')
plt.grid(None)

# Show the plot.
plt.show()

# Strong growth from 2014 onwards every year although slows down dramatically in 2019.
# Big decline in counts in 2019-2020 due to Covid-19 when lockdowns were in force.
# Decline not due necessarily due to actual reduction in cyclists.
# Counting had stopped through most of 2019-20 as displayed previously in missing data analysis.
# Post pandemic recovery in numbers still below 2015.
# Although 2021 numbers are not for full year, they do go up to end of first week of December 2021.
# Shows that there is certainly room to increase cycling uptake from current levels.
# Although expect that cycling counts should naturally creep up as post Covid normalisation continues.

## Micro Analysis of trends in London

### Cycling Trends in London by month

Pre-Covid

In [ ]:
# Filtering Pre-Covid Time period.
df1_precov = df1[(df1['year'] <2020)]

# Sorting according to the month, otherwise visualization allocates random postions to months.
df1_precov = df1_precov.sort_values(by=['month'], ascending=True)

In [ ]:
# Visualise change in uptake in the 3 parts of London over time.
plt.rcParams['figure.figsize']=12,5
g=sns.lineplot(x="month_name", y="log_total_uptake_ldn", data=df1_precov, hue='functional_cycling_area', 
               ci=None)
g.set_title('Cycling uptake in London & its regions')
g.set_ylabel('Cycling Uptake (log scale)')
g.set_xlabel('Month')
plt.legend(bbox_to_anchor=(-0.15, 0.1), borderaxespad=0, frameon=False)

plt.grid(b=None)


# Can see peak months are summer in all three areas of London.
# Inner & Outer London skewed by the fact most of the count takes place between Mar - Jul & Sep-Oct.
# But even here can see increase in uptake going into the peak summer months between May and Jul.

Cycling Trend during Covid (2020)

In [ ]:
# Filtering Covid Time period (2020).
df1_covid = df1[(df1['year'] == 2020)]

# Sorting according to the month, otherwise visualization allocates random postions to months.
df1_covid = df1_covid.sort_values(by=['month'], ascending=True)

In [ ]:
# Visualise change in uptake in the 3 parts of London over time.
g=sns.lineplot(x="month_name", y="log_total_uptake_ldn", data=df1_covid, hue='functional_cycling_area', 
               ci=None)
g.set_title('Cycling uptake in London & its regions')
g.set_ylabel('Cycling Uptake (log scale)')
g.set_xlabel('Month')
plt.legend(bbox_to_anchor=(-0.15, 0.1), borderaxespad=0, frameon=False)
plt.rcParams['figure.figsize']=(12,5)
plt.grid(b=None)

# Although monthly patterns from Pre-Covid holds.
# Trend here is likely linked to lock down periods.
# Can clearly see that there is very little data for Inner & Central London in 2020.
# Not very insightful for regional data.
# London needs to automate or improve its count methodology.
# How can New York count through COVID and not London??

Cycling Trend during Post-Covid (2021)

In [ ]:
# Filtering Covid Time period (2021).
df1_postcovid = df1[(df1['year'] == 2021)]

# Sorting values according to the month.
df1_postcovid = df1_postcovid.sort_values(by=['month'], ascending=True)

In [ ]:
# Visualise change in uptake in the 3 parts of London over time.
g=sns.lineplot(x="month_name", y="log_total_uptake_ldn", data=df1_postcovid, hue='functional_cycling_area', 
               ci=None)
g.set_title('Cycling uptake in London & its regions')
g.set_ylabel('Cycling Uptake (log scale)')
g.set_xlabel('Month')
plt.legend(bbox_to_anchor=(-0.15, 0.1), borderaxespad=0, frameon=False)
plt.rcParams['figure.figsize']=(12,5)
plt.grid(b=None)

# Data in Outer & Inner London still very patchy.
# No clear patterns here sas data is too patchy in the regions.

### Cycling Trends in London by season

In [ ]:
# Grouping by seasons to observe any patterns.
df1_seasons = df1.groupby(["season"], as_index=False)['total_uptake'].sum()

In [ ]:
# Plotting a treemap.
# Colour Palette.
colors=['#fae588','#f79d65','#f9dc5c','#e8ac65','#e76f51','#ef233c','#b7094c']

# Set Plot Style.
sns.set_style(style="whitegrid") 

# Setting the variable.
sizes= df1_seasons["total_uptake"].values
label=df1_seasons["season"]

# Setting the styles.
squarify.plot(sizes=sizes, label=label, alpha=0.6,color=colors).set(title='Cycling trends in London by season')
plt.axis('off')
plt.show()

# This will be somewhat skewed given Outer & Inner London data is\
# counted in just the Spring & Summer months.

In [ ]:
# Plotting an alternative visualisation.
plt.rcParams["axes.facecolor"] = "w"
sns.lineplot(data=df1_seasons, x="season", y="total_uptake", color="blue", markers=True, 
            ci=None)
plt.title("Cycling uptake in London by season")
plt.xlabel("Season")
plt.ylabel("Cycling uptake (log scale)")
plt.grid(None)
plt.show()

# Spring & Summer are peak 'seasons' for cycling in London.
# Cycling craters in the winter

### Cycling trends in London by weather & season

In [ ]:
# Grouping by weather type and months to observe any patterns.
df1_seasons = df1.groupby(["season", "weather"], as_index=False)['total_uptake'].sum()

In [ ]:
# Control figure size for this notebook.
plt.rcParams['figure.figsize'] = [8, 8]

# Data.
data = df1_seasons
 
# Use the scatterplot function to build the bubble map.
ax = sns.scatterplot(data=data, 
                x="season", 
                y="weather", 
                size="total_uptake", 
                legend=False, 
                sizes=(30, 1000))

ax.set_xlabel("Seasons")
ax.set_ylabel("Weather")
ax.set_title("Cycling trends in London during different weather conditions")

# Show the graph.
plt.show()

# Size of the bubbles indicates relative cycling uptake in each weather & season.
# Clear to see that irrespective Season, cycling uptake is highest during periods of 'good' weather.

### Cycling trend in London by Period of the Day

In [ ]:
# Function for assigning rank to Period of the day for sorting.
def f(x):
     if (x == "Early Morning (06:00-07:00)"):
            return '0'
    
     elif (x == "AM peak (07:00-10:00)"):
            return '1'
    
     elif (x == "Inter-peak (10:00-16:00)"):
            return'2'
    
     elif (x == "PM peak (16:00-19:00)"):
            return '3'
    
     elif (x == "Evening (19:00-22:00)"):
            return'4'

# Apply user defined function to create new column with seasons.
df1['period_rank'] = df1['period'].apply(f) 

In [ ]:
# Converting number of cycles into thousands for feasible graph comprehension.
df1['total_uptake'] = df1['total_uptake']/1000

# Sorting the values according to period rank.
# Otherwise Periods will appear at random on the graph.
df1 = df1.sort_values(by='period_rank', ascending=True)

ax = sns.catplot(data=df1, kind="bar", x= "total_uptake",y= "period", alpha=0.9, legend=True,
                 palette='dark', ci = None, hue = "functional_cycling_area", 
                height=6)

sns.set_theme(style="whitegrid")
plt.title("London cycling trends for period of the day (2015 - 2021)")
plt.xlabel("No. of cycles ( '000s )")
plt.ylabel("Period of the Day")
plt.xticks(rotation = 360)
plt.grid(None)
plt.show()

# Patterns across all 3 regions the same.
# AM Peak and PM Peak are the busiest periods.
# In Outer London Inter-peak is also another busy period.

### Cycling trend in London by Day of the Week

In [ ]:
# Function for assigning rank to:

def f(x):
     if (x == "Monday"):
            return '1'
    
     elif (x == "Tuesday"):
            return '2'
    
     elif (x == "Wednesday"):
            return'3'
    
     elif (x == "Thursday"):
            return '4'
    
     elif (x == "Friday"):
            return'5'

     elif (x == "Saturday"):
            return '6'
        
     elif (x == "Sunday"):
            return '7'
        
# Apply user defined function to create new column with day ranking (0 - 6).
df1['day_rank'] = df1['day_of_week'].apply(f) 

In [ ]:
# Sorting the values according to period rank.
# Otherwise Periods will appear at random on the graph.
df1 = df1.sort_values(by='day_rank', ascending=True)

In [ ]:
# Converting number of cycles into thousands for feasible graph comprehension.
df1['total_uptake'] = df1['total_uptake']/1000

# Setting out the plot.
ax = sns.catplot(data=df1, kind="bar", x= "total_uptake",y= "day_of_week", alpha=0.9, legend=True,
                 palette='dark', ci = None, hue = "functional_cycling_area", 
                height=6)

sns.set_theme(style="whitegrid")
plt.title("London cycling trends for day of the week (2015 - 2021)")
plt.xlabel("No. of cycles ( '000s )")
plt.ylabel("Day of the Week")
plt.xticks(rotation = 360)
plt.show()

# Cycling in Central London declines progressively through the week.
# Tues-Thurs are peak cycling days in Outer & Inner London.
# Almost no weekend cycling!
# No cycling on Sundays in Outer London at all??

### Cycling Trends in London by Ownership Patterns

In [ ]:
# Group ownership.
df = london_complete.groupby(["year", 
                              "functional_cycling_area"], 
                              as_index=False)["number_of_private_cycles",
                                              "number_of_cycle_hire_bikes"].sum()

In [ ]:
# Renaming Columns for easy comprehension and visualizations.
df = df.rename(columns={'number_of_private_cycles': 'Private', 
                        'number_of_cycle_hire_bikes': 'Hired'})

In [ ]:
# Drop outer as functional cycling area as the area does not have any ownership data.
df1 = df[df["functional_cycling_area"].str.contains("Central|Inner") == True]

In [ ]:
# Melt dataframe to get Private and Hired ownership in one column.
df1_melt = df1.melt(id_vars=["year", "functional_cycling_area"],
                              var_name="ownership",
                              value_name="count")

In [ ]:
# Set out the visualisation.
g = sns.catplot(x="year", 
                y="count", 
                hue="ownership", 
                kind = "bar", 
                col="functional_cycling_area", 
                data=df1_melt)

sns.set_theme(style="whitegrid")

plt.show()

# Private ownership overwhelmingly accounts for both absolute count & growth in count
# Almost no uptake in hired bikes in Inner
# Hired bikes is a big area of opportunity to increase cycling uptake

### Cycling Trends in London by Gender

In [ ]:
# Group year and cycling count by gender.
# Gender & cycling count data is in london_complete.
df = london_complete.groupby(["year"], 
                             as_index=False)["number_of_male_cycles",
                                             "number_of_female_cycles", 
                                             "number_of_unknown_cycles"].sum()

In [ ]:
# Renaming Columns for easy comprehension and visualizations.
df = df.rename(columns={'number_of_male_cycles': 'Male', 
                        'number_of_female_cycles': 'Female',
                        'number_of_unknown_cycles': 'Unknown'})

# Add the totals males and females.
df.loc['total'] = df.sum()

# View output.
df

In [ ]:
# Strip the decimal point from the year column.
df['year'] = df['year'].astype(str).str[:-2].astype(np.int64)
df

# Can see Males are 84% of total. 
# Females are 15% of total.
# Unknown 1% of total.

In [ ]:
# Calculate percentages.
total = 501584 + 90401 + 5198
male = 501584
female = 90401
unknown = 5198

percent_male = (male/total)*100
percent_female = (female/total)*100
percent_unknown = (unknown/total)*100

In [ ]:
# Plot Waffle Chart.
# Manually calculated % age of total from above dataframe.
data = {'Male': (percent_male), 'Female': (percent_female), 'Unknown': (percent_unknown)}
fig = plt.figure(FigureClass = Waffle, 
                 rows = 5, 
                 columns = 15, 
                 values = data, 
                 title = {'label': 'London Cyclists by Gender'}, 
                 legend={'loc': 'upper left', 'bbox_to_anchor': (1, 1)}, 
                 figsize = (10,15))

# Data is incomplete.
# Gender segregation only represented in Outer London Dataset.
# How do the counters know gender?
# Huge disparity in gender is very noteworth nevertheless!

In [ ]:
# Drop the total row.
df.drop(df.tail(1).index,inplace=True)

In [ ]:
# Investigating change in gender distribution Y-o-Y.
# Meling to get both genders in single column.
df_melt = df.melt(id_vars=["year"], 
                  var_name="gender", 
                  value_name="count")

In [ ]:
# See % change in counts amongst genders.
df_melt['pct_yoy'] = df_melt['count'].pct_change()*100

# View output randomly.
df_melt.sample(5)

In [ ]:
# Drop rows with 2014 data.
# Filtering Pre-Covid Time period.
df1 = df_melt[(df_melt['year'] >2014)]

# Drop rows with gender unknown.
df1 = df1[df1["gender"].str.contains("Unknown") == False]

In [ ]:
# Visualise Year on Year % Change. 
g = sns.catplot(x="year", y="pct_yoy", kind = "bar", col="gender", data=df1)

sns.set_theme(style="whitegrid")
plt.show()

# Very volatile Y-o-Y % change.
# No clear insight except that post Covid recovery amongst female riders is much stronger.
# More than double the rate of increase in male cyclists.
# Need to note that % change in women is from a very low base. 
# The stronger than male equivalent recovery should be used as a momentum builder?

In [ ]:
# Visualise absolute numbers across the years.
# Reducing the scalar effect.
df_melt['count'] = df_melt['count']/1000

# Setting out the plot.
ax = sns.catplot(data=df_melt, kind="bar", x= "year", y= "count", hue="gender")

sns.set_theme(style="whitegrid")
plt.title("Gender distribution of Outer London Cyclists (2015 - 2021)")
plt.ylabel("No. of cycles ( '000s )")
plt.xlabel("Year")
plt.xticks(rotation = 360)
plt.show()

# Once again confirms the massive gender gap in cycling uptake.

In [ ]:
# Cycling Infrastructure data exists only for 2017 & 2018.
# To compare linkage with infrastructure and gender drop all rows\
# for gender where year is not 2017 or 2018.
# Gender & cycling count data located in london_complete.
df = london_complete[(london_complete.year == 2017) | (london_complete.year == 2018)]

In [ ]:
# Filter out boroughs which collect data on gender. 
# Group  by year.  
df = df.groupby(["year", "borough"], as_index=False)["number_of_male_cycles",
                                                     "number_of_female_cycles", 
                                                     "number_of_unknown_cycles"].sum()

In [ ]:
# Renaming Columns for easy comprehension and visualizations.
df = df.rename(columns={'number_of_male_cycles': 'Male', 
                        'number_of_female_cycles': 'Female',
                       'number_of_unknown_cycles': 'Unknown'})

In [ ]:
# Drop boroughs which dont collect gender data.
df = df[~(df[['Male','Female','Unknown']] == 0).any(axis=1)]

In [ ]:
# Show how many boroughs collect gender data.
for col in df:
  print(col,": ", df[col].nunique())

# 23 Boroughs in London collect gender data in these two years.

In [ ]:
# Drop columns Male & Unknown.
df.drop(['Male', 'Unknown'], axis=1, inplace=True)

In [ ]:
# Set out the visualisation.
# Shows how the number of female cyclists have evolved over the two years of interest in each borough.
g = sns.catplot(x="Female", 
                y="borough", 
                hue="year", 
                kind = "bar", 
                data=df)

plt.title("Female Cyclists in London's Boroughs (2017-2018)")
plt.ylabel("Borough Name")
plt.xlabel("No. Female Cyclists")

sns.set_theme(style="whitegrid")
plt.show()

# Can see basically very little notable increase in most boroughs. 
# Except Ealing, Richmond upon Thames & Waltham Forest.
# In fact in some boroughs, numbers of female cyclists declined considerably.
# Such as in Brent, Greenwich, Harringey, Hillingdon, Hounslow, Lewisham, Merton & Sutton.
# Other boroughs where female cyclists are interestingly large are Kingston upon Thames & Newham.
# Worth looking at all these boroughs more closely and how they correspond with various infrastructure\ 
# and total infrastructure.
# So out of 23 London boroughs which collect gender data, we look at the 13 more closely\
# Ealing, Richmond upon Thames, Waltham Forest, Brent, Greenwich, Harringey, Hillingdon,\
# Hounslow, Lewisham, Merton, Sutton, Kingston upon Thames & Newham.

In [ ]:
# Group by Borough to get total number of female cyclist per borough.
df = df.groupby(["borough"], as_index=False)["Female"].sum()

In [ ]:
# Visualise the top 10 Boroughs for female cyclists.
df_largest_ten = df.nlargest(10,'Female')

df_largest_ten.plot.barh(x="borough", 
                         y="Female", 
                         title="Top 10 Boroughs for Female Cyclists 2017-2018", 
                         legend=False, 
                         xlabel='borough', 
                         ylabel='Female Cyclists')
plt.show()

### Investigating effect of each cycling infrastructure asset on female uptake

In [ ]:
# Fix the formatting of year column.
cyc_lane_yr_bor_grpby['year'] = cyc_lane_yr_bor_grpby['year'].astype(str).str[:-2].astype(np.int64)

In [ ]:
cyc_lane_yr_bor_grpby

In [ ]:
# Retain only 2017 & 2018 data in the cycling infrastructure data.
cyc_lane_yr_bor_grpby = cyc_lane_yr_bor_grpby[(cyc_lane_yr_bor_grpby.year == 2017) | (cyc_lane_yr_bor_grpby.year == 2018)]
rct_route_yr_bor_grpby = rct_route_yr_bor_grpby[(rct_route_yr_bor_grpby.year == 2017) | (rct_route_yr_bor_grpby.year == 2018)]
signage_yr_bor_grpby = signage_yr_bor_grpby[(signage_yr_bor_grpby.year == 2017) | (signage_yr_bor_grpby.year == 2018)]
signal_yr_bor_grpby = signal_yr_bor_grpby[(signal_yr_bor_grpby.year == 2017) | (signal_yr_bor_grpby.year == 2018)]
crossing_yr_bor_grpby = crossing_yr_bor_grpby[(crossing_yr_bor_grpby.year == 2017) | (crossing_yr_bor_grpby.year == 2018)]
cycle_park_yr_bor_grpby = cycle_park_yr_bor_grpby[(cycle_park_yr_bor_grpby.year == 2017) | (cycle_park_yr_bor_grpby.year == 2018)]
rct_point_yr_bor_grpby = rct_point_yr_bor_grpby[(rct_point_yr_bor_grpby.year == 2017) | (rct_point_yr_bor_grpby.year == 2018)]
traf_calm_yr_bor_grpby = traf_calm_yr_bor_grpby[(cyc_lane_yr_bor_grpby.year == 2017) | (traf_calm_yr_bor_grpby.year == 2018)]

In [ ]:
# Merging the female count data with cycling infrastructure.
df1 = df.merge(cyc_lane_yr_bor_grpby[['year', 'borough', 'infra_sum']])
df2 = df.merge(rct_route_yr_bor_grpby[['year', 'borough', 'infra_sum']])
df3 = df.merge(signage_yr_bor_grpby[['year', 'borough', 'infra_sum']])
df4 = df.merge(signal_yr_bor_grpby[['year', 'borough', 'infra_sum']])
df5 = df.merge(crossing_yr_bor_grpby[['year', 'borough', 'infra_sum']])
df6 = df.merge(cycle_park_yr_bor_grpby[['year', 'borough', 'infra_sum']])
df7 = df.merge(rct_point_yr_bor_grpby[['year', 'borough', 'infra_sum']])
df8 = df.merge(traf_calm_yr_bor_grpby[['year', 'borough', 'infra_sum']])

In [ ]:
# Grouping by Borough and summing the two variables.
df1 = df1.groupby(["borough"], as_index=False)["Female","infra_sum"].sum()
df2 = df2.groupby(["borough"], as_index=False)["Female","infra_sum"].sum()
df3 = df3.groupby(["borough"], as_index=False)["Female","infra_sum"].sum()
df4 = df4.groupby(["borough"], as_index=False)["Female","infra_sum"].sum()
df5 = df5.groupby(["borough"], as_index=False)["Female","infra_sum"].sum()
df6 = df6.groupby(["borough"], as_index=False)["Female","infra_sum"].sum()
df7 = df7.groupby(["borough"], as_index=False)["Female","infra_sum"].sum()
df8 = df8.groupby(["borough"], as_index=False)["Female","infra_sum"].sum()

In [ ]:
# Visualising Cycle Lanes & Female Cycling Uptake by boroughs.

# Use the scatterplot function to build the bubble map.

sns.scatterplot(data=df1, 
                x="infra_sum", 
                y="borough", 
                size="Female",
                hue="Female", 
                legend=True,
                sizes=(20, 2000), 
                palette= "tab10"
                )


plt.title('Cycle Lanes & Female Cycling Uptake by boroughs')
# Set x-axis label.
plt.xlabel('Number of Cycle Lanes')
# Set y-axis label.
plt.ylabel('Buroughs')

# Position of Legend.
plt.legend(bbox_to_anchor=(1.25, 0.5), loc='center right', borderaxespad=0, title= 'Female Cyclists')

# show the graph.
plt.show()

In [ ]:
# Visualising Restricted Routes & Female Cycling Uptake by boroughs.

# Use the scatterplot function to build the bubble map.

sns.scatterplot(data=df2, 
                x="infra_sum", 
                y="borough", 
                size="Female",
                hue="Female", 
                legend=True,
                sizes=(20, 2000), 
                palette= "tab10"
               )

plt.title('Restricted Routes & Female Cycling Uptake by boroughs')
# Set x-axis label.
plt.xlabel('Number of Restricted Routes')
# Set y-axis label.
plt.ylabel('Buroughs')

# Position of Legend.
plt.legend(bbox_to_anchor=(1.25, 0.5), loc='center right', borderaxespad=0, title= 'Female Cyclists')

# show the graph.
plt.show()

In [ ]:
# Visualising Signage & Female Cycling Uptake by boroughs.

# Use the scatterplot function to build the bubble map.

sns.scatterplot(data=df3, 
                x="infra_sum", 
                y="borough", 
                size="Female",
                hue="Female", 
                legend=True,
                sizes=(20, 2000), 
                palette= "tab10"
               )

plt.title('Signage & Female Cycling Uptake by boroughs')
# Set x-axis label.
plt.xlabel('Number of Signages')
# Set y-axis label.
plt.ylabel('Buroughs')

# Position of Legend.
plt.legend(bbox_to_anchor=(1.25, 0.5), loc='center right', borderaxespad=0, title= 'Female Cyclists')

# show the graph.
plt.show()

In [ ]:
# Visualising Signals & Female Cycling Uptake by boroughs.

# Use the scatterplot function to build the bubble map.

sns.scatterplot(data=df4, 
                x="infra_sum", 
                y="borough", 
                size="Female",
                hue="Female", 
                legend=True,
                sizes=(20, 2000), 
                palette= "tab10"
               )

plt.title('Signals & Female Cycling Uptake by boroughs')
# Set x-axis label.
plt.xlabel('Number of Signals')
# Set y-axis label.
plt.ylabel('Buroughs')

# Position of Legend.
plt.legend(bbox_to_anchor=(1.25, 0.5), loc='center right', borderaxespad=0, title= 'Female Cyclists')

# show the graph.
plt.show()

In [ ]:
# Visualising Crossing & Female Cycling Uptake by boroughs.

# Use the scatterplot function to build the bubble map.

sns.scatterplot(data=df5, 
                x="infra_sum", 
                y="borough", 
                size="Female",
                hue="Female", 
                legend=True,
                sizes=(20, 2000), 
                palette= "tab10"
               )

plt.title('Cycle Crossings & Female Cycling Uptake by boroughs')
# Set x-axis label.
plt.xlabel('Number of Crossings')
# Set y-axis label.
plt.ylabel('Buroughs')

# Position of Legend.
plt.legend(bbox_to_anchor=(1.25, 0.5), loc='center right', borderaxespad=0, title= 'Female Cyclists')

# show the graph.
plt.show()

In [ ]:
# Visualising Cycle Parkings & Female Cycling Uptake by boroughs.

# Use the scatterplot function to build the bubble map.

sns.scatterplot(data=df6, 
                x="infra_sum", 
                y="borough", 
                size="Female",
                hue="Female", 
                legend=True,
                sizes=(20, 2000), 
                palette= "tab10"
               )

plt.title('Cycle Parking & Female Cycling Uptake by boroughs')
# Set x-axis label.
plt.xlabel('Number of Cycle Parkings')
# Set y-axis label.
plt.ylabel('Buroughs')

# Position of Legend.
plt.legend(bbox_to_anchor=(1.25, 0.5), loc='center right', borderaxespad=0, title= 'Female Cyclists')

# Show the graph.
plt.show()

In [ ]:
# Visualising Restricted Points & Female Cycling Uptake by boroughs.

# Use the scatterplot function to build the bubble map.

sns.scatterplot(data=df7, 
                x="infra_sum", 
                y="borough", 
                size="Female",
                hue="Female", 
                legend=True,
                sizes=(20, 2000), 
                palette= "tab10"
               )

plt.title('Restricted Points & Female Cycling Uptake by boroughs')
# Set x-axis label.
plt.xlabel('Restricted Points')
# Set y-axis label.
plt.ylabel('Buroughs')

# Position of Legend.
plt.legend(bbox_to_anchor=(1.25, 0.5), loc='center right', borderaxespad=0, title= 'Female Cyclists')

# show the graph
plt.show()

In [ ]:
# Visualising Traffic Calming & Female Cycling Uptake by boroughs.

# Use the scatterplot function to build the bubble map.

sns.scatterplot(data=df8, x="infra_sum", 
                y="borough", 
                size="Female",
                hue="Female", 
                legend=True,
                sizes=(20, 2000), 
                palette= "tab10"
               )

plt.title('Traffic Calming & Female Cycling Uptake by boroughs')
# Set x-axis label.
plt.xlabel('Traffic Calming')
# Set y-axis label.
plt.ylabel('Buroughs')

# Position of Legend.
plt.legend(bbox_to_anchor=(1.25, 0.5), loc='center right', borderaxespad=0, title= 'Female Cyclists')

# show the graph.
plt.show()

In [ ]:
'''
# Visualising Restricted Routes & Female Cycling Uptake by boroughs
# Will use two horizontal bar charts side by side on a shared axis

# Flipping the values of the two variables to plot on shared axis


# Setting up an ordering for the shared axis
# Descending order set for boroughs vis a vis Total Infrastructure
boroughs = ['Croydon', 'Waltham Forest', 'Lambeth', 'Greenwich', 'Ealing', 
            'Barking & Dagenham', 'Enfield', 'Newham', 'Hounslow', 'Richmond upon Thames', 
            'Hillingdon', 'Harrow', 'Kingston upon Thames', 'Merton', 'Redbridge', 
            'Sutton', 'Haringey', 'Bromley', 'Havering', 'Brent', 'Lewisham', 
            'Bexley', 'Barnet']

# Setting up the plot area
sns.set_style(style='white')

# Setting up the two horizontal bar plots
ax1 = sns.barplot(x='infra_sum', y='borough', data=df2, order=boroughs, palette="Set2")
ax2 = sns.barplot(x='Female', y='borough', data=df2, order=boroughs, palette="Set2")
plt.title("Impact of Restricted Routes on Female Cycling Uptake in London 2017-2018")
plt.xlabel("               Female Cycling Uptake      /      Restricted Routes")



plt.show()

# Restricted Routes dont matter

In [ ]:
# Concatenating all cycling infrastructure data.
infra_complete=pd.concat([df1, df2, df3, df4, df5, df6, df7, df8])

# Adding up the total infrastructure in each borough of interest.
infra_complete1 = infra_complete.groupby(["borough"], 
                                         as_index=False)["infra_sum"].sum()

In [ ]:
# Visualise the top 10 Boroughs for total infrastructure.
infra_largest_ten = infra_complete1.nlargest(10,'infra_sum')

infra_largest_ten.plot.barh(x="borough", 
                            y="infra_sum", 
                            title="Top 10 Boroughs with Female Cyclists for Total Cycle Infrastructure 2017-2018", 
                            legend=False, 
                            xlabel='Total Infrastructure', 
                            ylabel='Borough Name' )
plt.show()

In [ ]:
# Merge Total Infrastructure with Female cyclist counts.
dfs = [infra_complete1,df]
on = ['borough']
gender_infra = ft.reduce(lambda left, right: pd.merge(left, right, on=on), dfs)

In [ ]:
# Sort by Female Cyclists.
gender_infra = gender_infra.sort_values('Female')

# Strip decimal points.
gender_infra['Female'] = gender_infra['Female'].astype(str).str[:-2].astype(np.int64)

In [ ]:
# Visualising Total Infrastructure of each Borough & Female Cycling Uptake.

# Use the scatterplot function to build the bubble map.

sns.scatterplot(data=df8, x="infra_sum", 
                y="borough", 
                size="Female",
                hue="Female", 
                legend=True,
                sizes=(20, 2000) 
                #palette= "black"
               )

plt.title('Total units of avaiable infrastructure assets', fontsize=23)
# Set x-axis label.
plt.xlabel('Total cycling infrastructure available in London', fontsize=18)
# Set y-axis label.
plt.ylabel('Boroughs', fontsize=18)

# Position of Legend.
plt.legend(bbox_to_anchor=(1.25, 0.5), loc='center right', borderaxespad=0, title= 'Female Cyclists')

# Show the graph
plt.show()

## Hired Bikes Analysis

This analysis is conducted to reveal insights through cycle hiring patterns and align strategy on cycling-specific infrastructure connectivity (hire bikes to broader transport network and reach total transport strategy - last mile)

In [ ]:
# Importing Dataset.
cycle_hire = pd.read_csv('TFL Cycle Hire 2017.csv')

In [ ]:
# Grouping dataset according to routes taken by cyclists.
cycle_hire_routes = cycle_hire.groupby(['StartStation Name', 
                                        'EndStation Name']).size().reset_index(name='Freq')

# Adding a column to allocate serial number to the routes.
cycle_hire_routes.insert(0, '#', range(1, 1 + len(cycle_hire_routes)))

# Calculating percetages of Frequecies of the routes
cycle_hire_routes['Freq %'] = (cycle_hire_routes['Freq'] / cycle_hire_routes['Freq'].sum()) * 100


# Sorting Dataset according to most frequented routes.
cycle_hire_routes = cycle_hire_routes.sort_values(by='Freq', ascending=False)

In [ ]:
# Viewing number of Unique start stations.
cycle_hire_start_stations = cycle_hire_routes['StartStation Name'].unique()

# Printing the names of Unique start stations.
print(cycle_hire_start_stations)

In [ ]:
# Viewing number of Unique start stations.
cycle_hire_end_stations = cycle_hire_routes['EndStation Name'].unique()

# Printing the names of Unique start stations.
print(cycle_hire_end_stations)

In [ ]:
# Visualizing ten most frequented Start Stations for Hired Bikes.

cycle_hire_start_stations_ten_largest = cycle_hire_routes.nlargest(10,'Freq')

cycle_hire_start_stations_ten_largest.plot.barh(x="StartStation Name", 
                                                y="Freq", 
                                                title="Top 10 Start Stations for Hired Bikes", 
                                                legend=False, 
                                                xlabel='Start Stations' , 
                                                ylabel='Frequency')
plt.show()

In [ ]:
# Visualizing ten most frequented Start Stations for Hired Bikes.

cycle_hire_end_stations_ten_largest = cycle_hire_routes.nlargest(10,'Freq')

cycle_hire_end_stations_ten_largest.plot.barh(x="EndStation Name", 
                                              y="Freq", 
                                              title="Top 10 end Stations for Hired Bikes", 
                                              legend=False, 
                                              xlabel='End Stations', 
                                              ylabel='No. of Trips')
plt.show()

## Using Heatmaps & Geospatial data to display time series

This section is intended to be technology demonstrator. If successful, the team will use this technology to build out key presentation topics for the final submission. For now, this is exploratory. Many of the varaibles were explored using this technology. For the purpose of this workbook one or two examples are being included by way of augmenting the analysis here as well as demonstrating the process and code behind the analysis using this tool which creates stunning visuals.

### Average number of cyclists throughout the day

In [ ]:
london_locations = london_complete[["site_id","location","latitude","longitude"]]
london_locations = london_locations.drop_duplicates()

london_period2 = london_complete.groupby(["site_id", "period","time"]).agg("sum").reset_index()
london_period2 = london_period2.sort_values(by="time",ascending=True)
london_period2 = london_period2[["site_id","period","total_uptake"]]

london_period2_data = pd.merge(london_period2,london_locations,how="left",on="site_id")

fig = px.density_mapbox(london_period2_data,
                        lat='latitude', 
                        lon='longitude', 
                        z='total_uptake', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="period",
                        zoom=0,
                        hover_name="location",
                        hover_data={"site_id":False,
                                    "period":True,
                                    "total_uptake":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Average number of cyclists throughout different times of the day",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# This shows the average flow of cyclists in all areas of London at any given time of the day

### Evolution of Cyclist Counts in London's Regions

**Inner London**

In [ ]:
london_locations = london_complete[["site_id","location","latitude","longitude"]]
london_locations = london_locations.drop_duplicates()

london_areas = london_complete.groupby(["year", "site_id", "functional_cycling_area"]).agg("sum").reset_index()
london_areas = london_areas[["year","site_id","functional_cycling_area","total_uptake"]]

london_area_data = pd.merge(london_areas,london_locations,how="left",on="site_id")

fig = px.density_mapbox(london_area_data[london_area_data["functional_cycling_area"]=="Inner"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_uptake', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"year":True,
                                    "site_id":False,
                                    "functional_cycling_area":True,
                                    "total_uptake":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year flow of Cyclist Counts in Inner london (2015-2021)",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

# Not all areas retain the same amount of cyclist count through the period

**Central London**

In [ ]:
fig = px.density_mapbox(london_area_data[london_area_data["functional_cycling_area"]=="Central"],
                        lat='latitude', 
                        lon='longitude', 
                        z='total_uptake', 
                        radius=10,
                        center=dict(lat=0, 
                                    lon=180), 
                        animation_frame="year",
                        zoom=0,
                        hover_name="location",
                        hover_data={"year":True,
                                    "site_id":False,
                                    "functional_cycling_area":True,
                                    "total_uptake":True,
                                   "location":True,
                                   "latitude":False,
                                   "longitude":False},
                                      title="Year-On-Year flow of Cyclist Counts in Central London (2014-2021)",
                                      height=1000,
                                      width=1000,
                                      opacity=1,
                                      mapbox_style="stamen-terrain")
fig.show()

**This format is very successful. However, given that these heatmaps are very resource intensive, the remainder of this work for the final submission will be switched to Tableau where the build-up & UX are more visually compelling whilst all of the analysis technology is retained.**

##### Quantitative Analysis

Although the data set is relatively small with just over 195 rows of data, it is worth running some simple quantitative analysis to investigate if traffic flow and car ownership have any influence on each other and more importantly on cycling uptake. They could point towards longer term policy potentials to influence and increase cycling uptake in London.

### Preparing the data for Quantitative Analysis

In [ ]:
# Getting the shape of traffic flow, Car ownership and cycle count organised. 
# Melt the dataframe on car ownership.
# This will align shape for further analysis.
# will categorise the boroughs into central, inner and outer london values.
df6 = pd.melt(car_own, id_vars =['year'], value_vars =['city_of_london', 'barking_and_dagenham', 'barnet', 'bexley', 
                                                      'brent', 'bromley', 'camden', 'croydon', 'ealing', 'enfield', 
                                                      'greenwich', 'hackney', 'hammersmith_and_fulham', 'haringey', 
                                                      'harrow', 'havering', 'hillingdon', 'hounslow', 'islington', 
                                                      'kensington_and_chelsea', 'kingston_upon_thames', 'lambeth', 
                                                      'lewisham', 'merton', 'newham', 'redbridge', 'richmond_upon_thames', 
                                                      'southwark', 'sutton', 'tower_hamlets', 'waltham_forest', 
                                                      'wandsworth', 'westminster'])

In [ ]:
# Rename variable to align to other datasets on london.
df6 = df6.rename(columns={'variable': 'borough', 'value': 'car_ownership_count'})

In [ ]:
# Melt the dataframe on traffic flows.
# This will align shape for further analysis.
# Will categorise the boroughs into central, inner and outer london values.

df7 = pd.melt(traffic_flow, id_vars =['year'], value_vars =['city_of_london', 'barking_and_dagenham', 'barnet', 'bexley', 
                                                      'brent', 'bromley', 'camden', 'croydon', 'ealing', 'enfield', 
                                                      'greenwich', 'hackney', 'hammersmith_and_fulham', 'haringey', 
                                                      'harrow', 'havering', 'hillingdon', 'hounslow', 'islington', 
                                                      'kensington_and_chelsea', 'kingston_upon_thames', 'lambeth', 
                                                      'lewisham', 'merton', 'newham', 'redbridge', 'richmond_upon_thames', 
                                                      'southwark', 'sutton', 'tower_hamlets', 'waltham_forest', 
                                                      'wandsworth', 'westminster'])

In [ ]:
# Rename variable to align to other datasets on london.
df7 = df7.rename(columns={'variable': 'borough', 'value': 'traffic_flow'})

In [ ]:
# Merge the traffic flow and car ownership details.
dfs = [df6, df7]
cols= ["year", "borough"]
other_stats = ft.reduce(lambda left, right: pd.merge(left, right, on=cols), dfs)

In [ ]:
# Now manipulate cycle count data in london and add to above dataframe.
# Reshape london df into a smaller df8 so its more resource efficient during manipulation.
df8 = london_complete.drop(['latitude', 'longitude', 'survey_date', 'number_of_male_cycles', 
                            'number_of_female_cycles', 'number_of_unknown_cycles', 'day_of_week', 
                           'month', 'number_of_private_cycles', 'season', 'number_of_private_cycles', 
                           'number_of_cycle_hire_bikes', 'total_outer', 'total_inn_cen', 'weather', 'period'], axis=1)

In [ ]:
# Align name of boroughs across df8 so that it can be merged. 
# Change content of borough column into lower case. 
df8['borough'] = df8['borough'].str.lower()
# Replace & with and
df8.replace({'borough': '&'}, {'borough': 'and'}, regex=True, inplace = True)
# Strip spaces with _
df8.replace({'borough': ' '}, {'borough': '_'}, regex=True, inplace = True)

In [ ]:
# Sort by year and borough to align to traffic and car ownership count data.
df8 = df8.groupby(['year', 'borough'], as_index=False)['total_uptake'].sum()
df8.sort_values(['borough'], inplace = True)

In [ ]:
# Merge the traffic flow and car ownership details.
dfs = [other_stats, df8]
cols= ["year", "borough"]
combined_stats_ldn = ft.reduce(lambda left, right: pd.merge(left, right, on=cols), dfs)

In [ ]:
# Strip non quantitative data columns.
# Will use these to test for correlation amongst numeric variable.
df10=combined_stats_ldn.drop(['year', 'borough'], axis=1)

In [ ]:
# Rename variable for clarity on visual analysis.
df10 = df10.rename(columns={'car_ownership_count': 'car ownership', 'traffic_flow': 'traffic flow',
                           'total_uptake': 'cycling uptake'})

### Conducting the analysis

In [ ]:
# Correlation of each variable.
# Setting up the correlation matrix.
plt.figure
sns.heatmap(df10.corr(),annot=True,cmap='coolwarm')
plt.title("Correlation Matrix between Car Ownership, Traffic Flow & Cycling Uptake", pad=20)
plt.show()

# Total traffic flow and car ownership in any location are strongly positively correlated.
# Implies an increase in one will lead to an increase in the other. 
# Correlation is not causation and requires further investigation.
# However, on the surface & intuitively these relationships make sense.
# Also note similar relationships identified globally in PESTLE.

In [ ]:
# Visualise sort of relationships if any between the numeric variablessns.set(style="ticks", color_codes=True).
sns.pairplot(df10)

# Setting up adjustments to the plot area.
plt.suptitle("Regression Test")
plt.subplots_adjust(top=0.9)

# Can visualise a reasonable linear relationship between traffic flow and car ownership.
# Visualisation doesnt suggest linear relationship between cycling up take and other variables.
# So although correlation matrix suggest very strong correlation between all three variables.
# Limited linear relationship established.
# Given the strength of the inverse correlation between cycling uptake and car ownership and traffic flow.
# Even though the relationship is not linear, car ownership and thus traffic flow should be targetted for reduction.
# This will improve cycling uptake.
# Evidence in PESTLE suggests non motorised zones in cities improves cycling uptake.
# This is often the corner stone of government policies around increasing cycling uptake.
# TW did not provide positive feedback on this work surprisingly.
# So this potential area of long term interest and one that is being evidently pursued in other cities is being dropped.
# This area of work can be picked up later and policy options explored using social analytics.

### Regression Analysis

Will help quantify visualisations in the pairplots above

Car Ownership & Traffic Flow

In [ ]:
# Set up a Simple Linear Regression Model.
# Dependent Variable.
y = df10['traffic flow']

# Independent Variable.
x = df10['car ownership']

# OLS model.
f = 'y ~ x'
test = ols(f, data=df10).fit()
print(test.summary())

# Pretty strong R2 suggesting 72% of changes in traffic flow are caused by changes in car ownership. 
# Prob F-stat is much smaller than the threshold which testifies that the variables are significant.
# Large value F Statistic also suggests that the null hypothesis can be rejected.
# p-value supports this.
# Meaning we can hypothesise statistically that traffic flow and car ownership affect each other.

Car Ownership & Cycling uptake

In [ ]:
# Set up a simple linear regression model.
# Dependent Variable.
y = df10['cycling uptake']

# Independent Variable.
x = df10['car ownership']

# OLS model.
f = 'y ~ x'
test = ols(f, data=df10).fit()
print(test.summary())

# Much lower R2 which was to be expected. 
# Non linear relationship between the variables was previously visualised.
# However F Statistic supports that a relationship exists between the variables.

Traffic Flow & Cycling uptake

In [ ]:
# Set up a simple linear regression model.
# Dependent Variable.
y = df10['cycling uptake']

# Independent Variable.
x = df10['traffic flow']

# OLS model.
f = 'y ~ x'
test = ols(f, data=df10).fit()
print(test.summary())

# Even lower R - squared but again output is expected.
# A non linear relationship between the variables was previously visualised.
# Not real world useful.
# F Statistic still significant enough to suggest a relationship between variables.

The quantitative analysis section is being dropped following lo-fi feedback from TW. 